In [1]:
import os, sys

import os, sys
print(os.path.abspath('..'))
sys.path.append(os.path.abspath('../scripts'))

import glob
import cv2
from skimage import data, filters, segmentation

from skimage import data
from skimage.filters.rank import entropy
from skimage.morphology import disk, ball, black_tophat, white_tophat
from skimage.measure import shannon_entropy, label, regionprops
from skimage.morphology import square, disk
from skimage.io import imread

from scipy.signal import find_peaks
import numpy as np

%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits
import math
import importlib
# importlib.reload(sys)

from tqdm.notebook import tqdm
import json


import concurrent.futures
from itertools import repeat
import multiprocessing

/auto/globalscratch/users/n/s/nsayez/deepsun_bioblue


In [2]:
import torch
!CUDA_VISIBLE_DEVICE=1
torch.cuda.is_available()

True

# Visualize the generated/predicted Masks 

In [3]:
def get_training_types(run_path):
    from omegaconf import OmegaConf
    cfg_dataset = OmegaConf.load(run_path/".hydra"/"config.yaml")["dataset"]
    types = cfg_dataset['train_dataset']['dtypes']
#     print(cfg_dataset)
#     print(types)
    return types

def get_training_strategy(run_path):
    from omegaconf import OmegaConf
    cfg_dataset = OmegaConf.load(run_path/".hydra"/"config.yaml")["dataset"]
    
    transforms = cfg_dataset['train_dataset']['transforms']
    
    last_transform = transforms[-1]
#     print(last_transform)
    
    strategy = ''
    if last_transform['_target_'] == 'bioblue.transforms.DeepsunRandomMaskSelector':
        strategy = "randMask"
    elif (last_transform['_target_'] == 'bioblue.transforms.DeepsunMaskMerger') and (
            len(cfg_dataset['train_dataset']['dtypes']) > 2):
        strategy = "mixedMask"
        
    return strategy

In [4]:
import ipywidgets as widgets

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, Normalize

from pathlib import Path

dataset_root = "/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation"
# dataset_root = "/globalscratch/users/n/s/nsayez/deepsun_bioblue/All/test_GT"

seg_types = ['GroundTruth_single']
st = 25
tophat_thresh= range(250, 500 + st, st)
gen_seg_types = [f'2023_T{t}_fgbg' for t in tophat_thresh]
print(gen_seg_types)
# gen_seg_types = [  
# #             'T350_canny',
#             'T400_canny',
# #             'T450_canny',
#             ]


image_folder     = os.path.join(dataset_root, 'image')
image_lst = sorted(glob.glob(os.path.join(image_folder, '**/*.FTS'), recursive=True))
print(len(image_lst))

gt_folder        = os.path.join(dataset_root, 'GroundTruth_single')
# gt_folder        = os.path.join(dataset_root, 'GroundTruth')
gt_lst = sorted(glob.glob(os.path.join(gt_folder, '*.png')))

generated_folders = {t : os.path.join(dataset_root, t) for t in gen_seg_types}
print(generated_folders)
generated_lsts = {t : sorted(glob.glob(os.path.join(generated_folders[t], '*.png'))) for t in gen_seg_types}
print(generated_lsts)


dir_lst = [
    # T500
    Path('/home/ucl/elen/nsayez/outputs/2023-01-19/23-41-01_2013-15_UNet_T500_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/00-37-54_2013-15_UNet_T500_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/03-38-03_2013-15_UNet_T500_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/05-39-51_2013-15_UNet_T500_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/19-11-13_2013-15_UNet_T500_StepLR_epoch_1_run5'),
    # T475
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/22-40-27_2013-15_UNet_T475_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/00-00-49_2013-15_UNet_T475_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/23-27-58_2013-15_UNet_T475_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/06-22-01_2013-15_UNet_T475_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/06-23-01_2013-15_UNet_T475_StepLR_epoch_1_run5'),
    # T450
    Path('/home/ucl/elen/nsayez/outputs/2023-01-19/23-43-05_2013-15_UNet_T450_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/02-11-27_2013-15_UNet_T450_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/04-29-47_2013-15_UNet_T450_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/07-03-59_2013-15_UNet_T450_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/19-11-13_2013-15_UNet_T450_StepLR_epoch_1_run5'),
    # T425
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/22-40-26_2013-15_UNet_T425_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/01-15-23_2013-15_UNet_T425_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/23-31-00_2013-15_UNet_T425_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/06-22-01_2013-15_UNet_T425_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/06-23-01_2013-15_UNet_T425_StepLR_epoch_1_run5'),
    # T400
    Path('/home/ucl/elen/nsayez/outputs/2023-01-19/23-46-12_2013-15_UNet_T400_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/02-16-31_2013-15_UNet_T400_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/04-43-27_2013-15_UNet_T400_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/07-23-05_2013-15_UNet_T400_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/19-11-11_2013-15_UNet_T400_StepLR_epoch_1_run5'),
    # T375
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/22-40-26_2013-15_UNet_T375_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/01-19-26_2013-15_UNet_T375_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/23-41-35_2013-15_UNet_T375_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/06-22-01_2013-15_UNet_T375_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/08-56-08_2013-15_UNet_T375_StepLR_epoch_1_run5'),
    # T350
    Path('/home/ucl/elen/nsayez/outputs/2023-01-19/23-49-10_2013-15_UNet_T350_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/02-27-09_2013-15_UNet_T350_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/04-46-59_2013-15_UNet_T350_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/07-48-16_2013-15_UNet_T350_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/21-19-16_2013-15_UNet_T350_StepLR_epoch_1_run5'),
    # T325
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/22-51-08_2013-15_UNet_T325_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/01-48-50_2013-15_UNet_T325_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/01-34-09_2013-15_UNet_T325_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/06-22-01_2013-15_UNet_T325_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-21/08-56-46_2013-15_UNet_T325_StepLR_epoch_1_run5'),
    # T300
    Path('/home/ucl/elen/nsayez/outputs/2023-01-19/23-50-28_2013-15_UNet_T300_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/03-08-40_2013-15_UNet_T300_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/04-52-01_2013-15_UNet_T300_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/07-50-47_2013-15_UNet_T300_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/outputs/2023-01-20/21-19-16_2013-15_UNet_T300_StepLR_epoch_1_run5'),
    
    
    # Multiple Thresholds
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/01-18-26_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run0'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/01-18-26_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run2'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/01-18-26_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run3'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/01-18-26_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run4'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/05-55-04_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run5'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/05-55-04_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run6'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/06-30-11_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run7'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/10-18-12_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run8'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/13-15-38_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run8'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/10-23-14_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run9'),
    Path('/home/ucl/elen/nsayez/bio-blueprints/outputs/2023-01-22/10-24-14_2013-15_UNet_T425_T375_T325_StepLR_epoch_1_run10'),

]

run_dirs = [Path(d) for d in dir_lst]
run_folders = { f"CNN-{rdir.stem[17:].replace('_StepLR_epoch_1_','_')}": str(rdir) for rdir in run_dirs}
# run_folders = { f"CNN-{rdir.stem[9:]}": str(rdir) for rdir in run_dirs}

# my_dir = '/globalscratch/users/n/s/nsayez'
# run_dirs = [
# #     Path(my_dir+'/outputs/2022-06-05/18-24-05'),
# #     Path(my_dir+'/outputs/2022-06-05/14-29-30'),
    
# #     #T400-T350
# #     Path(my_dir+"/outputs/2022-06-21/14-18-40"),
# #     #T400-T350_alternating
# #     Path(my_dir+"/outputs/2022-06-22/11-27-27"),
# ]

# run_folders = { f"CNN-{get_training_strategy(run_dir)}-{get_training_types(run_dir)[1:]}": str(run_dir) 
#                    for run_dir in run_dirs}

seg_folders = {t:generated_folders[t] if t in generated_folders.keys() else run_folders[t]+"/predictions"
                        for t in list(generated_folders.keys())+list(run_folders.keys())}

# num_TTA = [0,4]
num_TTA = [0]
pred_folders = {t+f'_{n}TTA': os.path.join(run_folders[t] , f'predictions_{n}TTA' if n > 0 else f'predictions_solDisk') 
# pred_folders = {t+f'_{n}TTA': os.path.join(run_folders[t] , f'predictions_{n}TTA' if n > 0 else f'predictions') 
                            for n in num_TTA
                            for t in run_folders.keys()}
pred_lsts = {t : sorted(glob.glob(os.path.join(pred_folders[t], '*.png'))) for t in pred_folders.keys()}
print()
print(pred_folders)
print()
# print(pred_lsts)

['2023_T250_fgbg', '2023_T275_fgbg', '2023_T300_fgbg', '2023_T325_fgbg', '2023_T350_fgbg', '2023_T375_fgbg', '2023_T400_fgbg', '2023_T425_fgbg', '2023_T450_fgbg', '2023_T475_fgbg', '2023_T500_fgbg']
36
{'2023_T250_fgbg': '/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation/2023_T250_fgbg', '2023_T275_fgbg': '/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation/2023_T275_fgbg', '2023_T300_fgbg': '/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation/2023_T300_fgbg', '2023_T325_fgbg': '/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation/2023_T325_fgbg', '2023_T350_fgbg': '/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation/2023_T350_fgbg', '2023_T375_fgbg': '/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation/2023_T375_fgbg', '2023_T400_fgbg': '/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation/2023_T400_fgbg', '2023_T425_fgbg': '/globalscratch/users/n/s/nsayez/deepsun_bioblue/ManualAnnotation/2023_T

In [5]:
max_idx = len(image_lst)
# 
# cmap_gen = cm.winter
cmap_gen = cm.turbo
cmap_gen = cmap_gen(range(255))
cmap_gen = ListedColormap([(0, 0, 0, 0), *cmap_gen])

cmap_pred = cm.cool
cmap_pred = cmap_pred(range(255))
cmap_pred = ListedColormap([(0, 0, 0, 0), *cmap_pred])


cmap_circle = cm.jet
cmap_circle = cmap_gen(range(255))
cmap_circle = ListedColormap([(0, 0, 0, 0), *cmap_circle])

cmap_gt = cm.Wistia
cmap_gt = cmap_gt(range(255))
cmap_gt = ListedColormap([(0, 0, 0, 0), *cmap_gt])

def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

def refresh(slider): 
    xlims0 = axes0[0].get_xlim()
    ylims0 = axes0[0].get_ylim()
    
    axes0[0].clear()
    axes0[0].axis('Off')
    
    
    hdulst:fits.HDUList = fits.open(image_lst[idx_slider.value])
    test_img = hdulst[0]
    header = test_img.header
    center = np.array(test_img.shape)//2
    radius = header['SOLAR_R']
    solar_disk = 1-create_circular_mask( test_img.shape[0], test_img.shape[1] ,center,radius)
#     test_img = imread(image_lst[idx_slider.value])
    
    
    gt_label = imread(gt_lst[idx_slider.value])
#     gen_label = imread(generated_lst[idx_slider.value])
    
#     axes0[0].set_title(os.path.basename(image_lst[idx_slider.value]))
    
    if img_cb.value:
        axes0[0].imshow(test_img.data, cmap="gray", interpolation="None")
        axes0[0].imshow(solar_disk, cmap=cmap_circle, interpolation="None",alpha=.5)
#         axes0[0].invert_yaxis()
        axes0[0].get_xlim()[::-1]
    if gt_cb.value:
        axes0[0].imshow(gt_label, cmap=cmap_gt, interpolation="None", alpha=.5)
    for i, cb in enumerate(gen_cbs):
        if cb.value:
            tmp = imread(generated_lsts[gen_seg_types[i]][idx_slider.value])
            cs = axes0[0].imshow(tmp, cmap=cmap_gen, interpolation="None", alpha=.7)
            
    for i, cb in enumerate(pred_cbs):
        if cb.value: 
            tmp = imread(pred_lsts[list(pred_folders.keys())[i]][idx_slider.value])
            axes0[0].imshow(tmp, cmap=cmap_pred, interpolation="None", alpha=.5)  
        
    if xlims0 != (0.0, 1.0):
        axes0[0].set_xlim(xlims0)
        axes0[0].set_ylim(ylims0)
        
    
    return

max_rows = 1
max_cols = 1

plt.ioff()
plt.style.use('default')
# plt.style.use('dark_background')
fig_widget0, axes0 = plt.subplots(nrows=max_rows, ncols=max_cols, figsize=(8,5))
# axes0.axis('Off')
try:
    len(axes0)
except TypeError:
    axes0 = [axes0]

plt.ion()
img_cb = widgets.Checkbox(value=True, description='img')
gt_cb = widgets.Checkbox(value=False, description='gt')
gen_cbs = [widgets.Checkbox(value=False, description=f'{t}') for t in gen_seg_types]

pred_cbs = [widgets.Checkbox(value=False, description=f'{t}') for t in pred_folders.keys()]
# print(pred_cbs)
# gen_cb = widgets.Checkbox(value=False, description='Show Generation')
idx_slider = widgets.IntSlider(value=0, min=0, max=max_idx-1, step=1, description="Image Index")


# Input image to predict
test_img = imread(image_lst[0])
#prediction
gen_label = imread(generated_lsts[gen_seg_types[0]][0])

# print(gen_cbs)
# print(test_img.dtype)
# print(gen_label.dtype)

axes0[0].set_title(os.path.basename(image_lst[0]))
axes0[0].imshow(test_img, cmap="gray", interpolation="None")

# fig_widget0.colorbar(cm.ScalarMappable(norm=Normalize(vmin=0, vmax=3, clip=False), cmap=cmap_gen), ax=axes0)
# fig_widget0.colorbar(cs)

img_cb.observe(refresh, names='value')
gt_cb.observe(refresh, names='value')
for cb in gen_cbs:
    cb.observe(refresh, names='value')
for cb in pred_cbs:
    cb.observe(refresh, names='value')
idx_slider.observe(refresh, names='value')

h_len = 3
vbox_gen = widgets.VBox([ widgets.HBox(gen_cbs[i*h_len:(i+1)*h_len]) for i in range(len(gen_seg_types)//h_len)])
vbox_pred = widgets.VBox([ widgets.HBox(pred_cbs[i*h_len:(i+1)*h_len]) for i in range(math.ceil(len(pred_cbs)/h_len))])
widgets.VBox([vbox_gen, vbox_pred, widgets.HBox([idx_slider,gt_cb]),  fig_widget0.canvas])

# Split sunspots into size categories



In [6]:
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

def conf_matrix(y_true, y_pred, labels=None, sample_weight=None, samplewise=False):
    MCM = multilabel_confusion_matrix(
                y_true,
                y_pred,
                sample_weight=sample_weight,
                labels=labels,
                samplewise=samplewise,
    )
    return MCM

def get_scores_dict(gt_label, pred_label, classes):
    ###### Separate penumbra and umbra classes
    perClassCM = conf_matrix(gt_label.flatten().cpu(),pred_label.flatten().cpu(), labels=fg_classes)
    
    CM_dict = {c: perClassCM[i] for i,c in enumerate(fg_classes)}
    IoU = [ CM_dict[k][1,1] / ( CM_dict[k][1,1] + CM_dict[k][0,1] + CM_dict[k][1,0] + 1e-5) for k in CM_dict]
    DSC = [ 2*CM_dict[k][1,1] / ( 2*CM_dict[k][1,1] + CM_dict[k][0,1] + CM_dict[k][1,0] + 1e-5) for k in CM_dict]
        
    scores_dict = {c: {"DICE": DSC[i], "IoU": IoU[i]} for i,c in enumerate(classes)}    
    return scores_dict

    
def crop_in(y_true, y_pred, bbox, padding):
        minX,maxX = max(bbox[0]-padding, 0), min( bbox[2]+padding, y_true.shape[0])
        minY,maxY = max(bbox[1]-padding, 0), min( bbox[3]+padding, y_true.shape[1])
        
        bbox2 = [minX,minY,maxX,maxY]

        deltasX = [bbox[0]-minX,bbox[2]-maxX]
        deltasY = [bbox[1]-minY,bbox[3]-maxY]
       
        new_gt_mask = np.zeros((maxX-minX,maxY-minY))
        new_gt_mask[deltasX[0]:deltasX[1],deltasY[0]:deltasY[1]] = y_true[bbox[0]:bbox[2], bbox[1]:bbox[3]]
        
        
        new_gen_mask = np.zeros((maxX-minX,maxY-minY))
        new_gen_mask[deltasX[0]:deltasX[1],deltasY[0]:deltasY[1]] = y_pred[bbox[0]:bbox[2], bbox[1]:bbox[3]]
        
        assert new_gt_mask.shape == new_gen_mask.shape
        
        return new_gt_mask, new_gen_mask
    

In [7]:
from matplotlib.patches import Rectangle
def show_bins(seg_type, bins, fg_bg, penumbra, umbra, metric='DICE'):
    for i in range(len(bins)):
        b = bins[i]

        scores_fg_bg = [item['value'] for item in fg_bg[i][metric]]
        scores_penumbra = [item['value'] for item in penumbra[i][metric]]
        scores_umbra = [item['value'] for item in umbra[i][metric]]

        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))  
        fig.suptitle(f"{seg_type}  : Sunspots of [{b[0]},{b[1]}] pixels")
        ax[0].hist(scores_fg_bg, bins=20)
        ax[0].set_xlim([0,1])
        ax[0].set_title(f"{metric} score \n FG vs BG", fontsize=8)
        ax[1].hist(scores_penumbra, bins=20)
        ax[1].set_xlim([0,1])
        ax[1].set_title(f"{metric} score \n Penumbra", fontsize=8)
        ax[2].hist(scores_umbra, bins=20)
        ax[2].set_xlim([0,1])
        ax[2].set_title(f"{metric} score \n Umbra", fontsize=8)
        
def show_bins2(seg_type, bins, fg_bg, penumbra, umbra, metric='DICE'):
    for i in range(len(bins)):
        b = bins[i]

        scores_fg_bg = [item['value'] for item in fg_bg[i][metric] if not item['details']['misdetected']]
        misdetections_fg_bg = [item['value'] for item in fg_bg[i][metric] if item['details']['misdetected']]
#         print(misdetections_fg_bg)
        scores_penumbra = [item['value'] for item in penumbra[i][metric]]
        scores_umbra = [item['value'] for item in umbra[i][metric]]

        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))  
        fig.suptitle(f"{seg_type}  : Sunspots of [{b[0]},{b[1]}] pixels")
#         ax[0].hist(misdetections_fg_bg, bins=1, color='r')
        ax[0].add_patch(Rectangle((-0.05, 0), 0.05, len(misdetections_fg_bg), color='r'))
        ax[0].hist(scores_fg_bg, bins=20)
        ax[0].set_xlim([-0.1,1])
        ax[0].set_title(f"{metric} score \n FG vs BG", fontsize=8)
    

In [8]:
def get_scores_dict(gt_label, pred_label, classes):
    ###### Separate penumbra and umbra classes
    
#     perClassCM = conf_matrix(gt_label.flatten(),pred_label.flatten(), labels=fg_classes)
    perClassCM = conf_matrix(gt_label.flatten(),pred_label.flatten(), labels=fg_classes)
    
#     perClassCM = conf_matrix(gt_label,pred_label, labels=fg_classes)
    CM_dict = {c: perClassCM[i] for i,c in enumerate(fg_classes)}
    
    IoU = [ CM_dict[k][1,1] / ( CM_dict[k][1,1] + CM_dict[k][0,1] + CM_dict[k][1,0] + 1e-5) for k in CM_dict]
#     print("IoU",IoU)
    
    DSC = [ 2*CM_dict[k][1,1] / ( 2*CM_dict[k][1,1] + CM_dict[k][0,1] + CM_dict[k][1,0] + 1e-5) for k in CM_dict]
#     print("DSC",DSC)
        
    scores_dict = {c: {"DICE": DSC[i], "IoU": IoU[i]} for i,c in enumerate(classes)}
#     scores_dict["foreground"]= {"DICE": DSC_fg_bg[0], "IoU": IoU_fg_bg[0]}
    
    return scores_dict

def find_bboxes2(labels):
    props_labels = regionprops(labels)
    
    bboxes = []
    masks = []
    for prop in props_labels:
#         if prop.area > 15:
        bbox = np.array(prop.bbox)
        bboxes.append(bbox)
        masks.append(prop.image)

    return bboxes , masks

def is_false_detection(det_msk, gt_label):
    intersection = (gt_label*det_msk).sum()
    return intersection == 0.0

def compute_false_positives_OLD(image_lst, gen_lst, device='cpu', bins=[], all_classes={'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}):
#     all_classes = {'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}
    
    bin2FP_dict = {c: {str(i): {'num_false_positives': 0,
                             'total_pixel_area': 0, 
                             "false_positives":[],
                             "true_positives":[],
                            }
                             for i in range(len(bins))}
                     for c,ids in all_classes.items()
                    }
    
    for i in tqdm(range(len(image_lst)),leave=False):
        basename = os.path.splitext(os.path.basename(image_lst[i]))[0]

        test_img = imread(image_lst[i])
        gt_label = imread(gt_lst[i])
        candidate_label = imread(gen_lst[i])
        
        for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)): # foreground, penumbra, umbra
                pbar.set_description(f"{c_name}")
                                     
                # Get the binary version of the masks 
                gt_class_fg, candidate_class_fg = get_class_binary_mask(gt_label, candidate_label, c_ids)

                # Get the regions (connected components) properties of masks
                label_im = label(gt_class_fg)
                props_bboxes, props_masks = find_bboxes2(label_im)

                label_gen = label(candidate_class_fg)
                props_bboxes_gen, props_masks_gen = find_bboxes2(label_gen)

                # Create isolated masks for each connected component
                whole_m2 = np.array([np.zeros_like(gt_class_fg) for item in props_masks_gen])
                for idx, b in enumerate(props_bboxes_gen):
                    whole_m2[idx][b[0]:b[2], b[1]:b[3]] = props_masks_gen[idx]
                                     
                for j in tqdm(range(len(props_bboxes_gen[:])),leave=False):
                    bbox = props_bboxes_gen[j]

                    cur_m_gen = whole_m2[j]
                    m_gen_area = np.sum(props_masks_gen[j])


                    cur_bin = -1
                    for b in bins:
                        if (b[0] <= m_gen_area ) and (b[1]>=m_gen_area):
                            cur_bin = bins.index(b)
                    
                    is_false = is_false_detection(cur_m_gen, gt_class_fg)
            
                    details = {"image": basename, 'bbox': bbox, 'false_positive': is_false, "area": m_gen_area }

                    if is_false:
                        bin2FP_dict[c_name][str(cur_bin)]['num_false_positives'] += 1
                        bin2FP_dict[c_name][str(cur_bin)]['total_pixel_area'] += m_gen_area

                        bin2FP_dict[c_name][str(cur_bin)]['false_positives'].append(details)
                    else:
                        bin2FP_dict[c_name][str(cur_bin)]['true_positives'].append(details)
            
    return bin2FP_dict


def compute_false_positives_img(i, image_lst, gt_lst, gen_lst, all_classes, bins):
    basename = os.path.splitext(os.path.basename(image_lst[i]))[0]

    test_img = imread(image_lst[i])
    gt_label = imread(gt_lst[i])
    candidate_label = imread(gen_lst[i])
    
    
    cur_FP_dict = {c:{str(b_i):{'num_false_positives': 0,
                             'total_pixel_area': 0, 
                             "false_positives":[],
                             "true_positives":[],
                            } for b_i in range(len(bins))} for c in all_classes}

    for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)): # foreground, penumbra, umbra
        pbar.set_description(f"{c_name}")

        # Get the binary version of the masks 
        gt_class_fg, candidate_class_fg = get_class_binary_mask(gt_label, candidate_label, c_ids)

        # Get the regions (connected components) properties of masks
        label_im = label(gt_class_fg)
        props_bboxes, props_masks = find_bboxes2(label_im)

        label_gen = label(candidate_class_fg)
        props_bboxes_gen, props_masks_gen = find_bboxes2(label_gen)

        # Create isolated masks for each connected component
        whole_m2 = np.array([np.zeros_like(gt_class_fg) for item in props_masks_gen])
        for idx, b in enumerate(props_bboxes_gen):
            whole_m2[idx][b[0]:b[2], b[1]:b[3]] = props_masks_gen[idx]

        for j in tqdm(range(len(props_bboxes_gen[:])),leave=False):
            bbox = props_bboxes_gen[j]

            cur_m_gen = whole_m2[j]
            m_gen_area = np.sum(props_masks_gen[j])


            cur_bin = -1
            for b in bins:
                if (b[0] <= m_gen_area ) and (b[1]>=m_gen_area):
                    cur_bin = bins.index(b)

            is_false = is_false_detection(cur_m_gen, gt_class_fg)

            details = {"image": basename, 'bbox': bbox, 'false_positive': is_false, "area": m_gen_area }

            if is_false:
#                 bin2FP_dict[c_name][str(cur_bin)]['num_false_positives'] += 1
#                 bin2FP_dict[c_name][str(cur_bin)]['total_pixel_area'] += m_gen_area
#                 bin2FP_dict[c_name][str(cur_bin)]['false_positives'].append(details)
                
                cur_FP_dict[c_name][str(cur_bin)]['num_false_positives'] += 1
                cur_FP_dict[c_name][str(cur_bin)]['total_pixel_area'] += m_gen_area
                cur_FP_dict[c_name][str(cur_bin)]['false_positives'].append(details)
                
                
            else:
#                 bin2FP_dict[c_name][str(cur_bin)]['true_positives'].append(details)
                cur_FP_dict[c_name][str(cur_bin)]['true_positives'].append(details)
                
    return cur_FP_dict
    


def compute_false_positives(image_lst, gen_lst, device='cpu', bins=[], all_classes={'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}):
#     all_classes = {'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}
#     print("here")
    bin2FP_dict = {c: {str(i): {'num_false_positives': 0,
                             'total_pixel_area': 0, 
                             "false_positives":[],
                             "true_positives":[],
                            }
                             for i in range(len(bins))}
                     for c,ids in all_classes.items()
                    }
    
    
    num_cpu = multiprocessing.cpu_count()
    with concurrent.futures.ProcessPoolExecutor(max_workers=int(np.floor(0.9*num_cpu))) as executor:
        
        for cur_FP_dict in tqdm(executor.map(compute_false_positives_img, 
                                                  range(len(image_lst)),
                                                  repeat(image_lst), repeat(gt_lst), repeat(gen_lst), 
                                                  repeat(all_classes), repeat(bins))):
            for c in all_classes:
                for b in range(len(bins)):
                    bin2FP_dict[c][str(b)]["num_false_positives"] += cur_FP_dict[c][str(b)]["num_false_positives"]
                    bin2FP_dict[c][str(b)]["total_pixel_area"] += cur_FP_dict[c][str(b)]["total_pixel_area"]
                    bin2FP_dict[c][str(b)]["false_positives"].extend(cur_FP_dict[c][str(b)]["false_positives"])
                    bin2FP_dict[c][str(b)]["true_positives"].extend(cur_FP_dict[c][str(b)]["true_positives"])
        
                    
    return bin2FP_dict

    
  
        

In [9]:
def find_closest_in_pred2(gt_m, gen_m_lst):

    intersections = np.repeat(gt_m[None,:,:], gen_m_lst.shape[0], axis=0)*gen_m_lst   

    sum_intersections = np.sum(intersections, axis=(1,2))

    top_idx = np.argmax(sum_intersections)
    top = np.max(sum_intersections)
   
    out_mask = None if top_idx == -1 else gen_m_lst[top_idx]

    return out_mask, top, top_idx


In [10]:
def get_gt_bin_sizes(image_lst, gt_lst, bins=[]):
    binned_sunspots = {i: [] for i in range(len(bins))}
    
    for i in tqdm(range(len(image_lst)),leave=False):
        basename = os.path.splitext(os.path.basename(image_lst[i]))[0]

#         test_img = imread(image_lst[i])
        gt_label = imread(gt_lst[i])
        
        gt_label_fg_bg = gt_label.copy()
        gt_label_fg_bg[gt_label_fg_bg>0] = 1
        
        label_im = label(gt_label_fg_bg)
        props_bboxes, props_masks = find_bboxes2(label_im)
        
        for j in tqdm(range(len(props_bboxes[:])),leave=False):
            bbox = props_bboxes[j]
            m = props_masks[j]
            
            area = np.sum(m)
            
            cur_bin = -1
            for b in bins:
                if (b[0] <= area ) and (b[1]>=area):
                    cur_bin = bins.index(b)
            
            details = {"image": basename, 'bbox': bbox}
            
            binned_sunspots[cur_bin].append(details)
    return binned_sunspots

In [11]:
def binning_sunspots3_OLD(image_lst, gen_lst, device='cpu', bins =[], all_classes={'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}):

#     bin2bbox_dict_umbra = {i: {'DICE':[], 'IoU':[]} for i in range(len(bins))}
#     bin2bbox_dict_penumbra = {i: {'DICE':[], 'IoU':[]} for i in range(len(bins))}
#     bin2bbox_dict_fg_bg = {i: {'DICE':[], 'IoU':[]} for i in range(len(bins))}

    bin2bbox_dict =  {c: {str(i): {'DICE':[], 'IoU':[]} for i in range(len(bins))}
                         for c,ids in all_classes.items()}
    
    for i in tqdm(range(len(image_lst)),leave=False):
        basename = os.path.splitext(os.path.basename(image_lst[i]))[0]
        test_b = basename
        gt_b = os.path.splitext(os.path.basename(gt_lst[i]))[0]
        gen_b = os.path.splitext(os.path.basename(gen_lst[i]))[0]
        assert test_b == gt_b == gen_b
        
        test_img = imread(image_lst[i])
        gt_label = imread(gt_lst[i])
        candidate_label = imread(gen_lst[i])
        
        for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)): # foreground, penumbra, umbra
            pbar.set_description(f"{c_name}")
            
            # Get the binary version of the masks 
            gt_class_fg, candidate_class_fg = get_class_binary_mask(gt_label, candidate_label, c_ids)
            
            # Get the regions (connected components) properties of masks
            label_im = label(gt_class_fg)
            props_bboxes, props_masks = find_bboxes2(label_im)

            label_gen = label(candidate_class_fg)
            props_bboxes_gen, props_masks_gen = find_bboxes2(label_gen)
        
            # Create isolated masks for each connected component
            whole_m2 = np.array([np.zeros_like(gt_class_fg) for item in props_masks_gen])
            for idx, b in enumerate(props_bboxes_gen):
                whole_m2[idx][b[0]:b[2], b[1]:b[3]] = props_masks_gen[idx]
            
            # Loop over all regions
            for j in tqdm(range(len(props_bboxes[:])),leave=False):
                bbox = props_bboxes[j]
                m = props_masks[j]

                area = np.sum(m)

                minX,maxX = max(bbox[0]-padding, 0), min( bbox[2]+padding, test_img.shape[0])
                minY,maxY = max(bbox[1]-padding, 0), min( bbox[3]+padding, test_img.shape[1])

                bbox2 = [minX,minY,maxX,maxY]
                
                # Identify the bins the region belongs to.
                cur_bin = -1
                for b in bins:
                    if (b[0] <= area ) and (b[1]>=area):
                        cur_bin = bins.index(b)
                        
                whole_m = np.zeros_like(gt_class_fg)
                whole_m[bbox[0]:bbox[2], bbox[1]:bbox[3]] = m
                
                if whole_m2.shape[0] != 0: # generated mask is not empty
                    # Find closest region in prediction/generated mask
                    m_closest_in_gen, match_intersection, match_idx = find_closest_in_pred2(
                                                                            whole_m, whole_m2)
                    if m_closest_in_gen is not None:
                        # Compute metric scores between gt and mask
                        union = np.logical_or(m_closest_in_gen, whole_m)
                        label_union = label(union)
                        union_b = regionprops(label_union)[0].bbox

                        reduced_m = whole_m[union_b[0]:union_b[2],
                                                          union_b[1]:union_b[3] ]
                        reduced_closest = m_closest_in_gen[union_b[0]:union_b[2],
                                                          union_b[1]:union_b[3] ]

                        score_dict_fg_bg = get_scores_dict(reduced_m, reduced_closest, [1]) 
#                         print(score_dict_fg_bg)
                        
#                        cur_score_dict = {k:v for k,v in score_dict.items()}
                        
                        cur_score_dict = {c_name:score_dict_fg_bg[1]}
                        
                    else: # No corresponding detection
                        cur_score_dict = {c_name: {"DICE": 0.0, "IoU": 0.0}}

                    details = {"image": basename, 'bbox': bbox2, 'misdetected': match_intersection == 0.0 }

                else:
                    cur_score_dict = {c_name: {"DICE": 0.0, "IoU": 0.0}}
                    details = {"image": basename, 'bbox': bbox2, 'misdetected': True }
                    
                    
                bin2bbox_dict[c_name][str(cur_bin)]['DICE'].append({'value': cur_score_dict[c_name]['DICE'],
                                                               'details': details})
                bin2bbox_dict[c_name][str(cur_bin)]['IoU'].append({'value': cur_score_dict[c_name]['IoU'],
                                                           'details': details})
 
    return     bin2bbox_dict


def get_class_binary_mask(gt_mask, candidate_mask ,class_ids):
    gt_label_class=np.zeros_like(gt_mask)
    for i in class_ids:
        gt_label_class[gt_mask == i] = 1
    
    candidate_label_class=np.zeros_like(gt_mask)
    for i in class_ids:
        candidate_label_class[candidate_mask == i] = 1
    
    return gt_label_class, candidate_label_class


def binning_sunspot_img_OLD(i, image_lst, gt_lst, gen_lst, all_classes, bins):
    basename = os.path.splitext(os.path.basename(image_lst[i]))[0]
    test_b = basename
    gt_b = os.path.splitext(os.path.basename(gt_lst[i]))[0]
    gen_b = os.path.splitext(os.path.basename(gen_lst[i]))[0]
    assert test_b == gt_b == gen_b

    test_img = imread(image_lst[i])
    gt_label = imread(gt_lst[i])
    candidate_label = imread(gen_lst[i])
    
    cur_DICE_IoU_dict = {c:{str(b_i):{"DICE":[],"IoU":[]} for b_i in range(len(bins))} for c in all_classes}

    for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)): # foreground, penumbra, umbra
        pbar.set_description(f"{c_name}")

        # Get the binary version of the masks 
        gt_class_fg, candidate_class_fg = get_class_binary_mask(gt_label, candidate_label, c_ids)

        # Get the regions (connected components) properties of masks
        label_im = label(gt_class_fg)
        props_bboxes, props_masks = find_bboxes2(label_im)

        label_gen = label(candidate_class_fg)
        props_bboxes_gen, props_masks_gen = find_bboxes2(label_gen)

        # Create isolated masks for each connected component
        whole_m2 = np.array([np.zeros_like(gt_class_fg) for item in props_masks_gen])
        for idx, b in enumerate(props_bboxes_gen):
            whole_m2[idx][b[0]:b[2], b[1]:b[3]] = props_masks_gen[idx]

        # Loop over all regions
        for j in tqdm(range(len(props_bboxes[:])),leave=False):
            bbox = props_bboxes[j]
            m = props_masks[j]

            area = np.sum(m)

            minX,maxX = max(bbox[0]-padding, 0), min( bbox[2]+padding, test_img.shape[0])
            minY,maxY = max(bbox[1]-padding, 0), min( bbox[3]+padding, test_img.shape[1])

            bbox2 = [minX,minY,maxX,maxY]

            # Identify the bins the region belongs to.
            cur_bin = -1
            for b in bins:
                if (b[0] <= area ) and (b[1]>=area):
                    cur_bin = bins.index(b)

            whole_m = np.zeros_like(gt_class_fg)
            whole_m[bbox[0]:bbox[2], bbox[1]:bbox[3]] = m

            if whole_m2.shape[0] != 0: # generated mask is not empty
                # Find closest region in prediction/generated mask
                m_closest_in_gen, match_intersection, match_idx = find_closest_in_pred2(
                                                                        whole_m, whole_m2)
                if m_closest_in_gen is not None:
                    # Compute metric scores between gt and mask
                    union = np.logical_or(m_closest_in_gen, whole_m)
                    label_union = label(union)
                    union_b = regionprops(label_union)[0].bbox

                    reduced_m = whole_m[union_b[0]:union_b[2],
                                                      union_b[1]:union_b[3] ]
                    reduced_closest = m_closest_in_gen[union_b[0]:union_b[2],
                                                      union_b[1]:union_b[3] ]

                    score_dict_fg_bg = get_scores_dict(reduced_m, reduced_closest, [1]) 
#                         print(score_dict_fg_bg)

#                        cur_score_dict = {k:v for k,v in score_dict.items()}

                    cur_score_dict = {c_name:score_dict_fg_bg[1]}

                else: # No corresponding detection
                    cur_score_dict = {c_name: {"DICE": 0.0, "IoU": 0.0}}

                details = {"image": basename, 'bbox': bbox2, 'misdetected': match_intersection == 0.0 }

            else:
                cur_score_dict = {c_name: {"DICE": 0.0, "IoU": 0.0}}
                details = {"image": basename, 'bbox': bbox2, 'misdetected': True }


#             bin2bbox_dict[c_name][str(cur_bin)]['DICE'].append({'value': cur_score_dict[c_name]['DICE'],
#                                                            'details': details})
#             bin2bbox_dict[c_name][str(cur_bin)]['IoU'].append({'value': cur_score_dict[c_name]['IoU'],
#                                                        'details': details})
            cur_DICE_IoU_dict[c_name][str(cur_bin)]['DICE'].append({'value': cur_score_dict[c_name]['DICE'],
                                                       'details': details})
            cur_DICE_IoU_dict[c_name][str(cur_bin)]['IoU'].append({'value': cur_score_dict[c_name]['IoU'],
                                                       'details': details})
    return cur_DICE_IoU_dict  

def binning_sunspot_img(i, image_lst, gt_lst, gen_lst, all_classes, bins):
    basename = os.path.splitext(os.path.basename(image_lst[i]))[0]
    test_b = basename
    gt_b = os.path.splitext(os.path.basename(gt_lst[i]))[0]
    gen_b = os.path.splitext(os.path.basename(gen_lst[i]))[0]
    assert test_b == gt_b == gen_b

    test_img = imread(image_lst[i])
    gt_label = imread(gt_lst[i])
    candidate_label = imread(gen_lst[i])
    
    cur_DICE_IoU_dict = {c:{str(b_i):{"DICE":[],"IoU":[]} for b_i in range(len(bins))} for c in all_classes}

    for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)): # foreground, penumbra, umbra
        pbar.set_description(f"{c_name}")

        # Get the binary version of the masks 
        gt_class_fg, candidate_class_fg = get_class_binary_mask(gt_label, candidate_label, c_ids)

        # Get the regions (connected components) properties of masks
        label_im = label(gt_class_fg)
        props_bboxes, props_masks = find_bboxes2(label_im)

        label_gen = label(candidate_class_fg)
        props_bboxes_gen, props_masks_gen = find_bboxes2(label_gen)

        # Create isolated masks for each connected component
        whole_m2 = np.array([np.zeros_like(gt_class_fg) for item in props_masks_gen])
        for idx, b in enumerate(props_bboxes_gen):
            whole_m2[idx][b[0]:b[2], b[1]:b[3]] = props_masks_gen[idx]

        # Loop over all regions
        for j in tqdm(range(len(props_bboxes[:])),leave=False):
            bbox = props_bboxes[j]
            m = props_masks[j]

            area = np.sum(m)

            minX,maxX = max(bbox[0]-padding, 0), min( bbox[2]+padding, test_img.shape[0])
            minY,maxY = max(bbox[1]-padding, 0), min( bbox[3]+padding, test_img.shape[1])

            bbox2 = [minX,minY,maxX,maxY]

            # Identify the bins the region belongs to.
            cur_bin = -1
            for b in bins:
                if (b[0] <= area ) and (b[1]>=area):
                    cur_bin = bins.index(b)

            whole_m = np.zeros_like(gt_class_fg)
            whole_m[bbox[0]:bbox[2], bbox[1]:bbox[3]] = m

            if whole_m2.shape[0] != 0: # generated mask is not empty
                # Find closest region in prediction/generated mask
                m_closest_in_gen, match_intersection, match_idx = find_closest_in_pred2(
                                                                        whole_m, whole_m2)
                if m_closest_in_gen is not None:
                    # Compute metric scores between gt and mask
                    union = np.logical_or(m_closest_in_gen, whole_m)
                    label_union = label(union)
                    union_b = regionprops(label_union)[0].bbox

                    reduced_m = whole_m[union_b[0]:union_b[2],
                                                      union_b[1]:union_b[3] ]
                    reduced_closest = m_closest_in_gen[union_b[0]:union_b[2],
                                                      union_b[1]:union_b[3] ]

                    score_dict_fg_bg = get_scores_dict(reduced_m, reduced_closest, [1]) 
#                         print(score_dict_fg_bg)

#                        cur_score_dict = {k:v for k,v in score_dict.items()}

                    cur_score_dict = {c_name:score_dict_fg_bg[1]}

                else: # No corresponding detection
                    cur_score_dict = {c_name: {"DICE": 0.0, "IoU": 0.0}}

                details = {"image": basename, 'bbox': bbox2, 'misdetected': match_intersection == 0.0 }

            else:
                cur_score_dict = {c_name: {"DICE": 0.0, "IoU": 0.0}}
                details = {"image": basename, 'bbox': bbox2, 'misdetected': True }

            cur_DICE_IoU_dict[c_name][str(cur_bin)]['DICE'].append({'value': cur_score_dict[c_name]['DICE'],
                                                       'details': details})
            cur_DICE_IoU_dict[c_name][str(cur_bin)]['IoU'].append({'value': cur_score_dict[c_name]['IoU'],
                                                       'details': details})
    return cur_DICE_IoU_dict  
    

def binning_sunspots3(image_lst, gen_lst, device='cpu', bins =[], all_classes={'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}):
    bin2bbox_dict =  {c: {str(i): {'DICE':[], 'IoU':[]} for i in range(len(bins))}
                         for c,ids in all_classes.items()}
    
    num_cpu = multiprocessing.cpu_count()
    with concurrent.futures.ProcessPoolExecutor(max_workers=int(np.floor(0.9*num_cpu))) as executor:
        
        for cur_img_bin_dict in tqdm(executor.map(binning_sunspot_img, 
                                                  range(len(image_lst)),
                                                  repeat(image_lst), repeat(gt_lst), repeat(gen_lst), 
                                                  repeat(all_classes), repeat(bins))):
            for c in all_classes:
                for b in range(len(bins)):
                    bin2bbox_dict[c][str(b)]["DICE"].extend(cur_img_bin_dict[c][str(b)]["DICE"])
                    bin2bbox_dict[c][str(b)]["IoU"].extend(cur_img_bin_dict[c][str(b)]["IoU"])
        
        
#         for i in tqdm(range(len(image_lst)),leave=False):
            
    return     bin2bbox_dict


In [12]:
padding = 5
fg_classes = [1]
bins = [[1,50],[51,100],[101,400],[401,800],[801,1200], [1201,2000], [2000,3000], [3000,np.inf]]

In [13]:
binned_gt = get_gt_bin_sizes(image_lst, gt_lst, bins)
  
{k: len(v) for k,v in binned_gt.items()}

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

{0: 315, 1: 90, 2: 114, 3: 51, 4: 29, 5: 16, 6: 3, 7: 0}

In [21]:
from scipy import ndimage 

def find_fusions(label_gt_m, label_gen_m):
    """
    Find the fusions between the ground truth and the generated mask.
    A fusion is defined as the number of connected components in the intersection of the two masks.
    label_gt_m: (1, H, W)
    label_gen_m: (N, H, W)
    """
#     print('find_fusions3')
#     print(label_gt_m.shape)
#     print(label_gen_m.shape)

    intersection = label_gen_m*label_gt_m
#     print('intersection',intersection.shape)

    count_fusions = [np.unique(cur_intersection) for cur_intersection in intersection]
    count_fusions = [cur_count_fusions[cur_count_fusions != 0] for cur_count_fusions in count_fusions]
    count_fusions = [len(cur_count_fusions) for cur_count_fusions in count_fusions]

    return count_fusions

def find_FN(label_gt_m, label_gen_m):
    """
    Find the False Positives in generated mask when compared with gound truth.
    label_gt_m: (N, H, W)
    label_gen_m: (1, H, W)
    """
#     print('find_FN')
#     print(label_gt_m.shape)
#     print(label_gen_m.shape)

    
    intersection = label_gen_m*label_gt_m
#     print('intersection',intersection.shape)

    FN = np.apply_over_axes(np.sum, intersection, [1,2]).squeeze()
#     print('FN',FN.shape)
    FN = (FN==0).astype(int)
#     print(FN)

    return FN

def find_FP(label_gt_m, label_gen_m):
    """
    Find the False Positives in generated mask when compared with gound truth.
    label_gt_m: (1, H, W)
    label_gen_m: (N, H, W)
    """
#     print('find_FP')
#     print(label_gt_m.shape)
#     print(label_gen_m.shape)

    
    intersection = label_gen_m*label_gt_m
#     print('intersection',intersection.shape)

    FP = np.apply_over_axes(np.sum, intersection, [1,2]).squeeze()
#     print('FP',FP.shape)
    FP = (FP==0).astype(int)
#     print(FP)

    return FP

def get_IoU_and_DICE_OLD(label_gt_m, label_gen_m):
    """
    Find the IoU and DICE between the ground truth and the generated mask.
    """
    intersection = label_gen_m*label_gt_m
    # print('intersection',intersection.shape)

    
    union_with_intersection = label_gen_m + label_gt_m
    # print('union',union.shape)
    union = (union_with_intersection) > 0

    DICE = np.apply_over_axes(np.sum, intersection, [1,2]).squeeze() / np.apply_over_axes(np.sum, union_with_intersection, [1,2]).squeeze()
    # print('DICE',DICE.shape)

    # intersection per prediction
    IoU = np.apply_over_axes(np.sum, intersection, [1,2]).squeeze()

    union = np.apply_over_axes(np.sum, union, [1,2]).squeeze()

    IoU = IoU/union


    return IoU, DICE

def get_IoU_and_DICE(label_gt_m, label_gen_m):
    """
    Find the IoU and DICE between the ground truth and the generated mask.
    """
    intersection = label_gen_m*label_gt_m
    
    # get the indices with non-zero intersection
    non_zero_intersections = np.max(intersection, axis=(1,2))
    A = np.argwhere(non_zero_intersections)

    if A.size == 0:
        return 0, 0
    
    A = A.squeeze()
        
    intersection = intersection[A,:,:]
    if intersection.ndim == 2:
        intersection = intersection[None,:,:]
    union_with_intersection = (label_gen_m + label_gt_m)[A,:,:]
    if union_with_intersection.ndim == 2:
        union_with_intersection = union_with_intersection[None,:,:]
    
    union = (union_with_intersection) > 0

    union_limits = np.max(union, axis=0)
    
    B = np.argwhere(union_limits)
    (ystart, xstart), (ystop, xstop) = B.min(0), B.max(0) + 1 

    union_with_intersection = union_with_intersection[:,ystart:ystop, xstart:xstop]
    union = union[:,ystart:ystop, xstart:xstop]
    intersection = intersection[:,ystart:ystop, xstart:xstop]

    DICE = np.apply_over_axes(np.sum, intersection, [1,2]).squeeze() / np.apply_over_axes(np.sum, union_with_intersection, [1,2]).squeeze()
    
    # intersection per prediction
    IoU = np.apply_over_axes(np.sum, intersection, [1,2]).squeeze()

    union = np.apply_over_axes(np.sum, union, [1,2]).squeeze()

    IoU = IoU/union


    return IoU, DICE

def counting_fusions_FP_FN_img(i, image_lst, gt_lst, gen_lst, all_classes, bins):
    basename = os.path.splitext(os.path.basename(image_lst[i]))[0]
    test_b = basename
    gt_b = os.path.splitext(os.path.basename(gt_lst[i]))[0]
    gen_b = os.path.splitext(os.path.basename(gen_lst[i]))[0]
    assert test_b == gt_b == gen_b

    test_img = imread(image_lst[i])
    gt_label = imread(gt_lst[i])
    candidate_label = imread(gen_lst[i])
    
    class_fp_fn_fusions = {c:{
                            'area':{},
                            'FP': {},
                            'FN': {},
                            'Fusions': {},
                            'IoU': {},
                            'DICE': {},      
                           } for c in all_classes}
    
    for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)): # foreground, penumbra, umbra
        pbar.set_description(f"{c_name}")

        # Get the binary version of the masks 
        gt_class_fg, candidate_class_fg = get_class_binary_mask(gt_label, candidate_label, c_ids)


        gt_label, num_gt_label = ndimage.label(gt_class_fg)
        gen_label, num_gen_label = ndimage.label(candidate_class_fg)
        
        
        whole_mask_gen = np.zeros((num_gen_label, gen_label.shape[0], gen_label.shape[1]))
        for idx, b in enumerate(np.unique(gen_label)[1:]):
            whole_mask_gen[idx,:,:] = gen_label == b
            
        whole_mask_gt = np.zeros((num_gt_label, gt_label.shape[0], gt_label.shape[1]))
        for idx, b in enumerate(np.unique(gt_label)[1:]):
            whole_mask_gt[idx,:,:] = gt_label == b
            
        
        fns     =      find_FN(whole_mask_gt[:,:], gen_label[None,:,:])
        fps     =      find_FP(gt_label[None,:,:], whole_mask_gen[:,:])
        fusions = find_fusions(gt_label[None,:,:], whole_mask_gen[:,:])
        
        if len(np.unique(gt_label)[1:])>1:
            bestIoUs, bestDICEs = [],[]
            for b in np.unique(gt_label)[1:]:
                cur_gt_label = gt_label == b
    
                cur_IoUs,cur_DICEs = get_IoU_and_DICE(cur_gt_label, whole_mask_gen[:,:])
                bestIoU, bestDICE = np.max(cur_IoUs),np.max(cur_DICEs)
                bestIoUs.append(bestIoU)
                bestDICEs.append(bestDICE)
            ious = np.array(bestIoUs)
            dices = np.array(bestDICEs)
            
#             ious  = [np.max(get_IoU((gt_label == b)[None,:,:], whole_mask_gen[:,:])) for b in np.unique(gt_label)[1:]]
#             dices = [np.max(get_DICE((gt_label == b)[None,:,:], whole_mask_gen[:,:])) for b in np.unique(gt_label)[1:]]
#             ious = np.array(ious)
#             dices = np.array(dices)
        else:
            ious = np.array([])
            dices = np.array([])
        
        
        centroids = np.array([ item.centroid for item in regionprops(gt_label)])
        areas   =      np.apply_over_axes(np.sum, whole_mask_gt, [1,2]).squeeze()
        
        class_fp_fn_fusions[c_name]['FP'] = fps
        class_fp_fn_fusions[c_name]['FN'] = fns
        class_fp_fn_fusions[c_name]['Fusions'] = fusions
        
        
        class_fp_fn_fusions[c_name]['area'] = areas
        class_fp_fn_fusions[c_name]['IoU'] = ious
        class_fp_fn_fusions[c_name]['DICE'] = dices
        
            
    return basename, class_fp_fn_fusions
    

def counting_fusions(image_lst, gen_lst, device='cpu', bins =[], all_classes={'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}):
    fp_fn_fusions_dict =  {c: {
#                             'FP': {},
#                             'FN': {},
#                             'Fusions': {}
                           }
                         for c,ids in all_classes.items()}
    
    num_cpu = multiprocessing.cpu_count()
    with concurrent.futures.ProcessPoolExecutor(max_workers=int(np.floor(0.9*num_cpu))) as executor:
        
        for bn, res_dict in tqdm(executor.map(counting_fusions_FP_FN_img, 
                                                  range(len(image_lst)),
                                                  repeat(image_lst), repeat(gt_lst), repeat(gen_lst), 
                                                  repeat(all_classes), repeat(bins))):
#                 print(bn,res_dict)
                for c in all_classes:
                    fp_fn_fusions_dict[c][bn] = {}
                    fp_fn_fusions_dict[c][bn]['area'] = res_dict[c]['area']
                
                    fp_fn_fusions_dict[c][bn]['FP'] = res_dict[c]['FP']
                    fp_fn_fusions_dict[c][bn]['FN'] = res_dict[c]['FN']
                    fp_fn_fusions_dict[c][bn]['Fusions'] = res_dict[c]['Fusions']
                    
                    fp_fn_fusions_dict[c][bn]['IoU'] = res_dict[c]['IoU']
                    fp_fn_fusions_dict[c][bn]['DICE'] = res_dict[c]['DICE']
        
        
            
    return fp_fn_fusions_dict
    

In [22]:
all_classes = {
                'foreground':[1,2],
#                 'penumbra':[1],
#                 'umbra':[2]
              }
fusions_per_generator = {t : counting_fusions(image_lst, generated_lsts[t], bins=bins, all_classes=all_classes)
                        for t in tqdm(gen_seg_types[2:])}

  0%|          | 0/9 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [33]:
regionprops()

def get_sunspot_locations(directory, img_name):
    
    gt_label = imread(gt_lst[i])
    candidate_label = imread(gen_lst[i])
    
    gt_class_fg, candidate_class_fg = get_class_binary_mask(gt_label, candidate_label, c_ids)
    
    props_labels = regionprops(gt_class_fg)
    gt_label, num_gt_label = ndimage.label(gt_class_fg)
    

for k in fusions_per_generator.keys():
    for img_k, img_dict in fusions_per_generator[k]['foreground'].items():
        print(img_k, img_dict)
    break
    


UPH20130113150018 {'area': array([4.030e+02, 3.170e+02, 1.300e+01, 3.250e+02, 5.900e+01, 2.729e+03,
       1.000e+00, 2.306e+03, 8.100e+01, 5.380e+02, 6.300e+01, 5.700e+01,
       2.126e+03, 1.340e+03, 4.100e+01, 1.300e+01, 1.400e+01]), 'FP': array([0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), 'FN': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), 'Fusions': [1, 1, 1, 0, 0, 0, 1, 0, 1, 2, 3, 1, 1, 1, 1, 1, 1, 0], 'IoU': array([0.8231441 , 0.85215054, 0.66666667, 0.84741144, 0.61111111,
       0.81348593, 0.        , 0.70801768, 0.02551834, 0.16130061,
       0.02028986, 0.66153846, 0.93375394, 0.84008683, 0.44318182,
       0.40625   , 0.        ]), 'DICE': array([0.45149701, 0.46008708, 0.4       , 0.45870206, 0.37931034,
       0.44857581, 0.        , 0.41452597, 0.02488336, 0.13889651,
       0.01988636, 0.39814815, 0.48287113, 0.45654738, 0.30708661,
       0.28888889, 0.        ])}
UPH20130210133630 {'area': array([ 209., 1084.,   76.,  526.,   70.,   15.]), 

In [59]:
# fusions_per_generator

{'2023_T500_fgbg': {'foreground': {'UPH20130113150018': {'area': array([4.030e+02, 3.170e+02, 1.300e+01, 3.250e+02, 5.900e+01, 2.729e+03,
           1.000e+00, 2.306e+03, 8.100e+01, 5.380e+02, 6.300e+01, 5.700e+01,
           2.126e+03, 1.340e+03, 4.100e+01, 1.300e+01, 1.400e+01]),
    'FP': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    'FN': array([0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]),
    'Fusions': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'IoU': array([0.72345679, 0.68769716, 0.        , 0.69325153, 0.        ,
           0.80252252, 0.        , 0.82721382, 0.2345679 , 0.70938628,
           0.56923077, 0.        , 0.8523048 , 0.65249814, 0.55769231,
           0.        , 0.        ]),
    'DICE': array([0.41977077, 0.40747664, 0.        , 0.40942029, 0.        ,
           0.44522191, 0.        , 0.45271868, 0.19      , 0.41499472,
           0.3627451 , 0.        , 0.46013205, 0.3948556 , 0.35802469,
           0.        , 0.        ])},
   'UPH20130210133630': {'are

In [26]:
all_classes = {
                'foreground':[1,2],
#                 'penumbra':[1],
#                 'umbra':[2]
              }
fusions_per_predictor = {t : counting_fusions(image_lst, pred_lsts[t], bins=bins, all_classes=all_classes)
                        for t in tqdm(list(pred_lsts.keys()) [:])}

  0%|          | 0/55 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
df = pd.DataFrame.from_dict(fusions_per_predictor)

In [14]:

all_classes = {
                'foreground':[1,2],
#                 'penumbra':[1],
#                 'umbra':[2]
              }
scores_per_generator = {t : binning_sunspots3(image_lst, generated_lsts[t], bins=bins, all_classes=all_classes)
                        for t in tqdm(gen_seg_types[:])}


  0%|          | 0/11 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [15]:
all_classes = {
                'foreground':[1,2],
#                 'penumbra':[1],
#                 'umbra':[2]
              }
FP_per_generator = {t : compute_false_positives(image_lst, generated_lsts[t], bins=bins, all_classes=all_classes) 
                    for t in tqdm(gen_seg_types[:])}

  0%|          | 0/11 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Remplir **scores_per_predictor** et **FP_per_predictor** à partir de rien.

In [32]:
scores_per_predictor = {t : binning_sunspots3(image_lst, pred_lsts[t], bins=bins, all_classes=all_classes)
                        for t in tqdm(list(pred_lsts.keys()) [:])}

  0%|          | 0/55 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [33]:
FP_per_predictor = {t : compute_false_positives(image_lst, pred_lsts[t], bins=bins) 
                    for t in tqdm(list(pred_lsts.keys()) [:])}

  0%|          | 0/55 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Ajouter des entrées à **scores_per_predictor** et **FP_per_predictor** déjà existants

In [83]:
tmp_scores_per_predictor = {t : binning_sunspots3(image_lst, pred_lsts[t], bins=bins)
                        for t in tqdm(list(pred_lsts.keys()) [9:])}
scores_per_predictor.update(tmp_scores_per_predictor)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

In [89]:
tmp_FP_per_predictor = {t : compute_false_positives(image_lst, pred_lsts[t], bins=bins) 
                    for t in tqdm(list(pred_lsts.keys()) [9:])}
FP_per_predictor.update(tmp_FP_per_predictor)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:01<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [100]:
# scores_per_generator
# FP_per_generator

In [18]:
print(len(FP_per_predictor.keys()))

45


In [28]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.int64):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.bool_):
            return bool(obj)
        return super(NpEncoder, self).default(obj)

In [24]:
with open('./2023_Fusions_per_generator_fgbg_solDisk.json', 'w') as f:
    json.dump(fusions_per_generator,f, cls=NpEncoder)

In [28]:
with open('./2023_Fusions_per_predictor_fgbg_solDisk.json', 'w') as f:
    json.dump(fusions_per_predictor,f, cls=NpEncoder)

In [15]:
with open('./2023_Fusions_per_generator_fgbg_solDisk.json', 'r') as f:
    fusions_per_generator = json.load(f)
with open('./2023_Fusions_per_predictor_fgbg_solDisk.json', 'r') as f:
    fusions_per_predictor =  json.load(f)

In [37]:
with open('./2023_scores_per_generator_fgbg_solDisk.json', 'w') as f:
    json.dump(scores_per_generator,f, cls=NpEncoder)

In [38]:
with open('./2023_scores_per_predictor_fgbg_solDisk.json', 'w') as f:
    json.dump(scores_per_predictor,f, cls=NpEncoder)

In [16]:
with open('./2023_scores_per_generator_fgbg_solDisk.json', 'r') as f:
    scores_per_generator = json.load(f)
with open('./2023_scores_per_predictor_fgbg_solDisk.json', 'r') as f:
    scores_per_predictor = json.load(f)

In [40]:
with open('./2023_FPs_per_generator_fgbg_solDisk.json', 'w') as f:
    json.dump(FP_per_generator,f, cls=NpEncoder)

In [41]:
with open('./2023_FPs_per_predictor_fgbg_solDisk.json', 'w') as f:
    json.dump(FP_per_predictor,f, cls=NpEncoder)

In [17]:
with open('./2023_FPs_per_generator_fgbg_solDisk.json', 'r') as f:
    FP_per_generator = json.load(f)
with open('./2023_FPs_per_predictor_fgbg_solDisk.json', 'r') as f:
    FP_per_predictor = json.load(f)

In [18]:
score_per_segmenter = {"generators":scores_per_generator, "predictors": scores_per_predictor}
FP_per_segmenter = {"generators":FP_per_generator, "predictors": FP_per_predictor}

print(score_per_segmenter["generators"].keys())
print(score_per_segmenter["predictors"].keys())


dict_keys(['2023_T250_fgbg', '2023_T275_fgbg', '2023_T300_fgbg', '2023_T325_fgbg', '2023_T350_fgbg', '2023_T375_fgbg', '2023_T400_fgbg', '2023_T425_fgbg', '2023_T450_fgbg', '2023_T475_fgbg', '2023_T500_fgbg'])
dict_keys(['CNN-UNet_T500_run0_0TTA', 'CNN-UNet_T500_run2_0TTA', 'CNN-UNet_T500_run3_0TTA', 'CNN-UNet_T500_run4_0TTA', 'CNN-UNet_T500_run5_0TTA', 'CNN-UNet_T475_run0_0TTA', 'CNN-UNet_T475_run2_0TTA', 'CNN-UNet_T475_run3_0TTA', 'CNN-UNet_T475_run4_0TTA', 'CNN-UNet_T475_run5_0TTA', 'CNN-UNet_T450_run0_0TTA', 'CNN-UNet_T450_run2_0TTA', 'CNN-UNet_T450_run3_0TTA', 'CNN-UNet_T450_run4_0TTA', 'CNN-UNet_T450_run5_0TTA', 'CNN-UNet_T425_run0_0TTA', 'CNN-UNet_T425_run2_0TTA', 'CNN-UNet_T425_run3_0TTA', 'CNN-UNet_T425_run4_0TTA', 'CNN-UNet_T425_run5_0TTA', 'CNN-UNet_T400_run0_0TTA', 'CNN-UNet_T400_run2_0TTA', 'CNN-UNet_T400_run3_0TTA', 'CNN-UNet_T400_run4_0TTA', 'CNN-UNet_T400_run5_0TTA', 'CNN-UNet_T375_run0_0TTA', 'CNN-UNet_T375_run2_0TTA', 'CNN-UNet_T375_run3_0TTA', 'CNN-UNet_T375_run4_0TT

In [19]:
def compare_generators_stats(generators_names, scores_per_generator, FP_per_generator, metric='DICE',
                            all_classes = {'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}):
#     all_classes = {'foreground':[1,2], 'penumbra':[1], 'umbra':[2]}
    
    out_dict = {c: {i:{} for i in range(len(bins))} for c in all_classes.keys()}
    
#     print(scores_per_generator)
    
    for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)):
        
        for i, b in enumerate( bins[:]):   
            scores = {gen_name: [item['value'] 
                                           for item in scores_per_generator[gen_name][c_name][str(i)][metric]] 
                                for gen_name in generators_names}
            
            for i_t in range(len(generators_names)):
                gen_name = generators_names[i_t]

                scores_fg_bg = [item['value'] 
                                    for item in scores_per_generator[gen_name][c_name][str(i)][metric] 
                                if not item['details']['misdetected']]

                misdetections_fg_bg = [item['value'] 
                                           for item in scores_per_generator[gen_name][c_name][str(i)][metric] 
                                       if item['details']['misdetected']]

                
                false_positives = FP_per_generator[gen_name][c_name][str(i)]["num_false_positives"]
                false_positives_area = FP_per_generator[gen_name][c_name][str(i)]["total_pixel_area"]
        
                missed = len(misdetections_fg_bg)
        
                tmp_scores =  np.array(scores_fg_bg)
                mu, sigma = scipy.stats.norm.fit(tmp_scores)
                
                FP = false_positives
                FN = missed
                # TP = GT - FN
                TP = len(binned_gt[i]) - FN
                
#                 print(f'TP:{TP}, FP:{FP}, FN:{TP}')
                F1 = 2 * (TP / (2*TP + FP + FN)) if len(binned_gt[i]) > 0 else None

                out_dict[c_name][i][gen_name] = {
                            "F1": F1,
                            "num_FN": missed,
                            "percent_FN": 100* missed / len(binned_gt[i]) if len(binned_gt[i]) > 0 else None ,
                            "num_FP": false_positives,
                            "percent_FP": 100* false_positives / len(binned_gt[i]) if len(binned_gt[i]) > 0 else None, 
                            "FP_area": false_positives_area
                            }
                
    return out_dict
    
    ################
#     out_dict = {i:{} for i in range(len(bins))}
    

#     for i, b in enumerate( bins[:]):
            
#         scores_fg_bg = {gen_name: [item['value'] 
#                                        for item in scores_per_generator[gen_name][0][str(i)][metric]] 
#                                                                     for gen_name in generators_names}
#         for i_t in range(len(generators_names)):
#             gen_name = generators_names[i_t]

#             scores_fg_bg = [item['value'] 
# #                                 for item in scores_per_generator[gen_name][0][i][metric] 
#                                 for item in scores_per_generator[gen_name][0][str(i)][metric] 
#                             if not item['details']['misdetected']]

#             misdetections_fg_bg = [item['value'] 
#                                        for item in scores_per_generator[gen_name][0][str(i)][metric] 
# #                                        for item in scores_per_generator[gen_name][0][i][metric] 
#                                    if item['details']['misdetected']]
        
        
#             false_positives = FP_per_generator[gen_name][str(i)]["num_false_positives"]
#             false_positives_area = FP_per_generator[gen_name][str(i)]["total_pixel_area"]
# #             false_positives = FP_per_generator[gen_name][i]["num_false_positives"]
# #             false_positives_area = FP_per_generator[gen_name][i]["total_pixel_area"]
            
#             missed = len(misdetections_fg_bg)
        
#             tmp_scores =  np.array(scores_fg_bg)
#             mu, sigma = scipy.stats.norm.fit(tmp_scores)
            
#             out_dict[i][gen_name] = {
#                         "num_FN": missed,
#                         "percent_FN": 100* missed / len(binned_gt[i]) if len(binned_gt[i]) > 0 else None ,
#                         "num_FP": false_positives,
#                         "percent_FP": 100* false_positives / len(binned_gt[i]) if len(binned_gt[i]) > 0 else None, 
#                         "FP_area": false_positives_area
#                         }
    
#     return out_dict

In [21]:
tmp = [False,False,False,False,False,
#        True,
       False,]

True in tmp

False

In [20]:
import scipy
import pandas as pd
from IPython.display import display

# seg_to_compare = [
#     'CNN-Unet_TTA_T400_T350_Alternating_ConstantLR_30epochs_0TTA',
#     'CNN-Unet_TTA_T400_T350_Alternating_ConstantLR_30epochs_4TTA',
#     'CNN-Unet_TTA_T400_T350_canny_Alternating_MultistepLR_60epochs_0TTA',
#     'CNN-Unet_TTA_T400_T350_canny_Alternating_MultistepLR_60epochs_4TTA',
#     'CNN-Unet_TTA_T400_T350_canny_Alternating_StepLR_60epochs_0TTA', 
#     'CNN-Unet_TTA_T400_T350_canny_Alternating_StepLR_60epochs_4TTA', 
#     'CNN-TransUnet_TTA_T400_T350_canny_Alternating_StepLR_100epochs_0TTA',
#     'CNN-TransUnet_TTA_T400_T350_canny_Alternating_StepLR_100epochs_4TTA'
# ]
seg_to_compare = gen_seg_types

int_format = ["num_FN", "num_FP", "FP_area"]
float_format = ["percent_FN","percent_FP"]

all_classes = {'foreground':[1,2], 
#                'penumbra':[1], 
#                'umbra':[2]
              }
bins_stats = compare_generators_stats(seg_to_compare, score_per_segmenter["generators"], FP_per_segmenter["generators"], metric="DICE", all_classes=all_classes)
# bins_stats = compare_generators_stats(seg_to_compare, score_per_segmenter["predictors"], FP_per_segmenter["predictors"], metric="DICE", all_classes=all_classes)

# print(bins_stats)

all_classes = {
    'foreground':[1,2], 
#     'penumbra':[1],
#     'umbra':[2]
              }
for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)):
    print(c_name,c_ids)
    for i in range(len(bins)):
    #     print(bins[i])
        df = pd.DataFrame.from_dict(bins_stats[c_name][i])#.astype(float)
    #     display(df)
    
        
        #F1 
        F1 = df.transpose()["F1"].to_numpy()
    
        # Roc curves
        FN_rate = df.transpose()["percent_FN"].to_numpy()
        print(FN_rate)
        FP_rate = df.transpose()["percent_FP"].to_numpy()
        
        #plot FP and FN rates along their index in a single plot, add a legend, set ticks vertically
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
        ax.set_title(f"Bin of sunspots of size  {bins[i]} -> {len(binned_gt[i])} in GroudTruth")
        ax.plot( seg_to_compare, FN_rate*0.01, marker='o')
        ax.plot( seg_to_compare, FP_rate*0.01, marker='o')
        ax.plot( seg_to_compare, F1, marker='o')
        ax.legend(['FN_rate', 'FP_rate', 'F1 score'])
        ax.set_ylim([0,1])
        # set x_label vertically
        for tick in ax.get_xticklabels():
            tick.set_rotation(90)
        fig.tight_layout()
        
            
#         fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10,5))
#         ax[0].plot( seg_to_compare, FN_rate, marker='.')
#         ax[0].set_title("FN rate")
#         ax[0].set_ylim([0,100])
#         for tick in ax[0].get_xticklabels():
#             tick.set_rotation(90)
#         ax[1].plot( seg_to_compare, FP_rate, marker='.')
#         ax[1].set_title("FP rate")
#         ax[1].set_ylim([0,100])
#         for tick in ax[1].get_xticklabels():
#             tick.set_rotation(90)
#         plt.show()


        for row in df.index:
            if row in float_format:
                df.loc[row] = df.loc[row].map('{:,.2f}'.format)
            elif row in int_format:
                df.loc[row] = df.loc[row].map('{:,.0f}'.format)

    #     df.loc["percent_missed"] = pd.Series(100* df.loc["num_missed"] / len(binned_gt[i]), dtype=int)

        styler = df.transpose().style
    #     pd.set_option('display.float_format',
    #       lambda x: '{:,0f}'.format(x) if( (abs(x) % 1) != 0 )else '{:,.0f}'.format(x))
        styles = [dict(selector="caption", props=[("text-align", "center"),
                                              ("font-size", "120%"),
                                              ("font-weight", "bold")])]
        styler.set_caption(f"Bin of sunspots of size  {bins[i]} -> {len(binned_gt[i])} in GroudTruth").set_table_styles(styles)
        styler.set_properties(**{'text-align': 'center'})
        display(styler)
    #     display(df.transpose())
    

    

  0%|          | 0/1 [00:00<?, ?it/s]

/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:409: RuntimeWarning: Mean of empty slice.
  loc = data.mean()
/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:414: RuntimeWarning: Mean of empty slice.
  scale = np.sqrt(((data - loc)**2).mean())


  0%|          | 0/1 [00:00<?, ?it/s]

foreground [1, 2]
[26.34920635 33.01587302 41.26984127 51.74603175 56.82539683 63.80952381
 68.88888889 72.6984127  75.87301587 80.         84.44444444]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,F1,num_FN,percent_FN,num_FP,percent_FP,FP_area
2023_T250_fgbg,0.857143,0,0.00,1,33.33,"2,617"
2023_T275_fgbg,1.000000,0,0.00,0,0.00,0
2023_T300_fgbg,1.000000,0,0.00,0,0.00,0
2023_T325_fgbg,1.000000,0,0.00,0,0.00,0
2023_T350_fgbg,1.000000,0,0.00,0,0.00,0
2023_T375_fgbg,1.000000,0,0.00,0,0.00,0
2023_T400_fgbg,1.000000,0,0.00,0,0.00,0
2023_T425_fgbg,1.000000,0,0.00,0,0.00,0
2023_T450_fgbg,1.000000,0,0.00,0,0.00,0
2023_T475_fgbg,1.000000,0,0.00,0,0.00,0


[nan nan nan nan nan nan nan nan nan nan nan]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,F1,num_FN,percent_FN,num_FP,percent_FP,FP_area
2023_T250_fgbg,nan,0,nan,1,nan,"3,142"
2023_T275_fgbg,nan,0,nan,0,nan,0
2023_T300_fgbg,nan,0,nan,0,nan,0
2023_T325_fgbg,nan,0,nan,0,nan,0
2023_T350_fgbg,nan,0,nan,0,nan,0
2023_T375_fgbg,nan,0,nan,0,nan,0
2023_T400_fgbg,nan,0,nan,0,nan,0
2023_T425_fgbg,nan,0,nan,0,nan,0
2023_T450_fgbg,nan,0,nan,0,nan,0
2023_T475_fgbg,nan,0,nan,0,nan,0


In [42]:
import seaborn as sns
import re 
def compute_fps_fusions_matches(big_dict, gen, c_name):
#     print(big_dict[gen][c_name])
    FPs = []
    FNs = []
    Matches = []
    Fusions = []
    Fusions_V2 = []
    
    for img_key, img_stats in big_dict[gen][c_name].items():
#         print(img_stats)
        cur_img_FP = np.sum(img_stats['FP'])
        cur_img_FN = np.sum(img_stats['FN'])
        cur_img_Fusions = np.sum([v for v in img_stats['Fusions'] if v > 1])
        cur_img_Fusions_V2 = np.sum([1 for v in img_stats['Fusions'] if v > 1])
        cur_img_Matches = np.sum([1 for v in img_stats['Fusions'] if v == 1])

#         cur_img_FP = np.sum([1 for k,v in img_fusions.items() if v == 0])
#         cur_img_Matches = np.sum([1 for k,v in img_fusions.items() if v == 1])
#         cur_img_Fusions = np.sum([v for k,v in img_fusions.items() if v > 1])
        
        FPs.append(cur_img_FP)
        FNs.append(cur_img_FN)
        Matches.append(cur_img_Matches)
        Fusions.append(cur_img_Fusions)
        Fusions_V2.append(cur_img_Fusions_V2)
        
        
    return {'tot_FPs': np.sum(FPs), 'tot_FNs': np.sum(FNs), 'tot_Matches': np.sum(Matches), 
            'tot_Fusions': np.sum(Fusions), 'tot_Fusions_V2': np.sum(Fusions_V2),
           'FPs': FPs, 'Matches': Matches, 'Fusions': Fusions, 'Fusions_V2': Fusions_V2}
        

names_gen = []
th_vals_gen = []
num_th_gen = []
for seg in fusions_per_generator.keys():
#     print(seg)
    reg_finder = re.search('2023_(.*)_fgbg', seg)
    names_gen.append(f'{reg_finder.group(1).split("_")}')
    
    reg_finder2 = re.findall('T([0-9]+)', reg_finder.group(1))
    th_vals_gen.append([int(x) for x in reg_finder2])
    num_th_gen.append(len(reg_finder2))
# print(names_gen, th_vals)

names_pred = []
th_vals_pred = []
run_numbers_pred = []
num_ttas_pred = []
num_th_pred = []
for seg in fusions_per_predictor.keys():
#     print(seg)
    reg_finder = re.search('CNN-UNet_(.*)_run(.+)_(.)TTA', seg)
    names_pred.append(f'{reg_finder.group(1).split("_")}')
    run_numbers_pred.append(reg_finder.group(2))
    num_ttas_pred.append(reg_finder.group(3))
    
    reg_finder2 = re.findall('T([0-9]+)', reg_finder.group(1))
    th_vals_pred.append([int(x) for x in reg_finder2])
    num_th_pred.append(len(reg_finder2))
# print(num_th)
# print(names_gen, th_vals)

new_stats_gen = { k: compute_fps_fusions_matches(fusions_per_generator, k, 'foreground') for k, v  in fusions_per_generator.items()}
new_stats_pred = { k: compute_fps_fusions_matches(fusions_per_predictor, k, 'foreground') for k, v  in fusions_per_predictor.items()}


df_fus_miss_gen = pd.DataFrame(new_stats_gen).transpose()
df_fus_miss_pred = pd.DataFrame(new_stats_pred).transpose()

for f in ['tot_FPs','tot_FNs','tot_Fusions','tot_Fusions_V2','tot_Matches']:
    df_fus_miss_gen[f]=df_fus_miss_gen[f].astype(int)
    df_fus_miss_pred[f]=df_fus_miss_pred[f].astype(int)
    
df_fus_miss_gen['name'] = names_gen
df_fus_miss_pred['name'] = names_pred
df_fus_miss_pred['run_number'] = run_numbers_pred
df_fus_miss_pred['num_ttas'] = num_ttas_pred
df_fus_miss_pred['num_th'] = num_th_pred

df_fus_miss_pred = df_fus_miss_pred.groupby(['name']).mean()

# df2.append(pd.Series([int(t[0]) for t in th_vals], index=df.columns, name='th0_val'))
print(df_fus_miss_gen)
print(df_fus_miss_pred)

# print(df_fus_miss.info())
# print(new_stats)
fig_fus_miss, ax_fus_miss = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
sns.set_style("darkgrid")
g = sns.lineplot(
            data=df_fus_miss_gen,
            x="tot_FNs", y="tot_Fusions_V2",
            ax=ax_fus_miss,
    )
sns.scatterplot(data=df_fus_miss_gen,
                x='tot_FNs', y='tot_Fusions_V2', 
#                 hue='name', 
#                 style='name',
                s=150
               )

g = sns.lineplot(
            data=df_fus_miss_pred.loc[df_fus_miss_pred['num_th'] == 1],
            x="tot_FNs", y="tot_Fusions_V2",
            ax=ax_fus_miss,
    )
sns.scatterplot(data=df_fus_miss_pred.loc[df_fus_miss_pred['num_th'] == 1],
                x='tot_FNs', y='tot_Fusions_V2', 
#                 hue='name', 
#                 style='name',
                s=150,
                marker='X'
               )

# g = sns.lineplot(
#             data=df_fus_miss_pred.loc[df_fus_miss_pred['num_th'] == 3],
#             x="tot_FNs", y="tot_Fusions_V2",
#             ax=ax_fus_miss,
#     )
sns.scatterplot(data=df_fus_miss_pred.loc[df_fus_miss_pred['num_th'] == 3],
                x='tot_FNs', y='tot_Fusions_V2', 
#                 hue='name', 
#                 style='name',
                s=150,
                marker='s'
               )


fig_fus_miss.tight_layout()

                tot_FPs  tot_FNs  tot_Matches  tot_Fusions  tot_Fusions_V2  \
2023_T250_fgbg     3417       89          355          196              73   
2023_T275_fgbg     1277      112          355          178              68   
2023_T300_fgbg      566      138          352          154              61   
2023_T325_fgbg      276      174          342          133              54   
2023_T350_fgbg      121      196          327          120              52   
2023_T375_fgbg       69      221          327          105              47   
2023_T400_fgbg       37      242          323           91              40   
2023_T425_fgbg       18      260          321           77              34   
2023_T450_fgbg        8      279          308           65              29   
2023_T475_fgbg        5      299          294           52              23   
2023_T500_fgbg        2      320          295           32              15   

                                                              F

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [158]:
import re
seg_to_compare = [k for k,v in score_per_segmenter['predictors'].items()]
print(seg_to_compare)

# seg_to_compare = ['CNN-UNet_T400_T350_run0_0TTA']
# print(seg_to_compare)
# print(re.search('CNN-UNet_(.*)_run(.)_(.)TTA', seg_to_compare[-1]).group(1).split("_"))
# print(re.search('CNN-UNet_(.*)_run(.)_(.)TTA', seg_to_compare[-1]).group(2).split("_"))
# show_names = [f'CNN-{re.search("CNN-2013-15_UNet_(.*)_StepLR*", seg).group(1).split("_")}'.replace(" ","") for seg in seg_to_compare]
# print(show_names)

show_names = []
run_numbers = []
num_ttas = []
th_vals = []
for seg in seg_to_compare:
    print(seg)
    reg_finder = re.search('CNN-UNet_(.*)_run(.+)_(.)TTA', seg)
    show_names.append(f'CNN-{reg_finder.group(1).split("_")}')
    run_numbers.append(reg_finder.group(2).split("_"))
    num_ttas.append(reg_finder.group(3).split("_"))
    
    reg_finder2 = re.findall('T([0-9]+)', reg_finder.group(1))
    th_vals.append([int(x) for x in reg_finder2])

    
print(show_names)
print(run_numbers)
print(num_ttas)
print(th_vals)
    
    


['CNN-UNet_T500_run0_0TTA', 'CNN-UNet_T500_run2_0TTA', 'CNN-UNet_T500_run3_0TTA', 'CNN-UNet_T500_run4_0TTA', 'CNN-UNet_T500_run5_0TTA', 'CNN-UNet_T475_run0_0TTA', 'CNN-UNet_T475_run2_0TTA', 'CNN-UNet_T475_run3_0TTA', 'CNN-UNet_T475_run4_0TTA', 'CNN-UNet_T475_run5_0TTA', 'CNN-UNet_T450_run0_0TTA', 'CNN-UNet_T450_run2_0TTA', 'CNN-UNet_T450_run3_0TTA', 'CNN-UNet_T450_run4_0TTA', 'CNN-UNet_T450_run5_0TTA', 'CNN-UNet_T425_run0_0TTA', 'CNN-UNet_T425_run2_0TTA', 'CNN-UNet_T425_run3_0TTA', 'CNN-UNet_T425_run4_0TTA', 'CNN-UNet_T425_run5_0TTA', 'CNN-UNet_T400_run0_0TTA', 'CNN-UNet_T400_run2_0TTA', 'CNN-UNet_T400_run3_0TTA', 'CNN-UNet_T400_run4_0TTA', 'CNN-UNet_T400_run5_0TTA', 'CNN-UNet_T375_run0_0TTA', 'CNN-UNet_T375_run2_0TTA', 'CNN-UNet_T375_run3_0TTA', 'CNN-UNet_T375_run4_0TTA', 'CNN-UNet_T375_run5_0TTA', 'CNN-UNet_T350_run0_0TTA', 'CNN-UNet_T350_run2_0TTA', 'CNN-UNet_T350_run3_0TTA', 'CNN-UNet_T350_run4_0TTA', 'CNN-UNet_T350_run5_0TTA', 'CNN-UNet_T325_run0_0TTA', 'CNN-UNet_T325_run2_0TTA', 

In [193]:
import scipy
import pandas as pd
from IPython.display import display
import seaborn as sns

gen_to_compare = [k for k,v in score_per_segmenter['generators'].items()]
bins_stats_gen = compare_generators_stats(gen_to_compare, score_per_segmenter["generators"], FP_per_segmenter["generators"], metric="DICE", all_classes=all_classes)
names_gen = []
th_vals_gen = []
num_th_gen = []
for seg in fusions_per_generator.keys():
#     print(seg)
    reg_finder = re.search('2023_(.*)_fgbg', seg)
    names_gen.append(f'{reg_finder.group(1).split("_")}')
    
    reg_finder2 = re.findall('T([0-9]+)', reg_finder.group(1))
    th_vals_gen.append([int(x) for x in reg_finder2])
    num_th_gen.append(len(reg_finder2))
print(names_gen)
    

seg_to_compare = [k for k,v in score_per_segmenter['predictors'].items()]
show_names = []
th_vals = []
run_numbers = []
num_ttas = []
for seg in seg_to_compare:
    reg_finder = re.search('CNN-UNet_(.*)_run(.+)_(.)TTA', seg)
    show_names.append(f'CNN-{reg_finder.group(1).split("_")}')
    run_numbers.append(reg_finder.group(2))
    num_ttas.append(reg_finder.group(3))
    
    reg_finder2 = re.findall('T([0-9]+)', reg_finder.group(1))
    th_vals.append([int(x) for x in reg_finder2])

num_TH = [len(n.split(',')) for n in show_names]
print(run_numbers)
print(num_TH)
print(show_names)
    
index_lists = [[index for (index, item) in enumerate(num_TH) if item == i] for i in [1,2,3]]
print(index_lists)

int_format = ["num_FN", "num_FP", "FP_area"]
float_format = ["percent_FN","percent_FP"]

all_classes = {
    'foreground':[1,2], 
#     'penumbra':[1],
#     'umbra':[2]
              }

bins_stats_pred = compare_generators_stats(seg_to_compare, score_per_segmenter["predictors"], FP_per_segmenter["predictors"], metric="DICE",all_classes=all_classes)

# print(bins_stats_pred)


for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)):
    print(c_name,c_ids)
    for i in range(len(bins)):
        df = pd.DataFrame.from_dict(bins_stats_pred[c_name][i])#.astype(float)
        
        df2 = df.append(pd.Series(show_names, index=df.columns, name='run_name'))
        df2 = df2.append(pd.Series(run_numbers, index=df.columns, name='run_number'))
        df2 = df2.append(pd.Series(num_TH, index=df.columns, name='num_th'))
        df2 = df2.append(pd.Series(num_ttas, index=df.columns, name='num_tta'))
        df2 = df2.append(pd.Series([int(np.median(t)) for t in th_vals], index=df.columns, name='th0_val'))
        
        df2 = df2.transpose()
        for f in int_format:
            df2[f]=df2[f].astype(int)
        for f in float_format:
            df2[f]=df2[f].astype(float)
        for f in ['F1','th0_val','num_tta','num_th','run_number']:
            df2[f] = pd.to_numeric(df2[f])
        df2.index = range(len(th_vals))
        df2["percent_FN"] = 0.01*df2["percent_FN"]
        df2["percent_FP"] = 0.01*df2["percent_FP"]
        
#         print(df2)
        
        
        #F1 
        F1 = df.transpose()["F1"].to_numpy()
    
        # Roc curves
        FN_rate = df.transpose()["percent_FN"].to_numpy()
#         print(FN_rate)
        FP_rate = df.transpose()["percent_FP"].to_numpy()
    
        # show the F1 scores of single-thresholds
        df3 = df2.loc[df2['num_th'] == 1]
         
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
        g = sns.lineplot(
                    data=df3,
                    x="th0_val", y="F1",
#                     markers=["o"], 
#                     kind="line",
                    label='F1',
                    ax=ax[0]
                )
        g = sns.lineplot(
                    data=df3,
                    x="th0_val", y="percent_FN",
#                     markers=["o"], 
#                     kind="line",
                    label='FN',
                    ax=ax[0]
            )
        g = sns.lineplot(
                    data=df3,
                    x="th0_val", y="percent_FP",
#                     markers=["o"], 
#                     kind="line",
                    label='FP',
                    ax=ax[0]
            )
        g.set_ylim([0,1.1])
        
        ########## Compare Thresholding with CNN-1threshold
    
        g = sns.lineplot(
                    data=df3,
                    x="th0_val", y="F1",
#                     markers=["o"], 
#                     kind="line",
                    ax=ax[1],
                    label='CNN-1threshold'
                )
    
        
        df_gen = pd.DataFrame.from_dict(bins_stats_gen[c_name][i]) 
        df_gen = df_gen.transpose() 
        df_gen['name'] = names_gen
        df_gen['th0_val'] = [np.median(t) for t in th_vals_gen]
        
        for f in int_format:
            df_gen[f]=df_gen[f].astype(int)
        for f in float_format:
            df_gen[f]=df_gen[f].astype(float)
            
#         print(df_gen)
    
        g = sns.lineplot(
            data=df_gen,
            x="th0_val", y="F1",
            ax=ax[1],
            label='Thresholding'
        )
        g.set_ylim([0,1.1])
        
        ######## Add CNN-Multithreshold
        
        df4 = df2.loc[df2['num_th'] > 1].drop(['B', 'C'], axis=1)
#         df4['th_med'] = th0_val
        df4 = df4.loc[df4['run_number'] > 3]
        
#         print(df4)
        df4 = df4.groupby(['run_name']).mean()  
#         print(df4)
        
        g = sns.lineplot(
                    data=df4,
                    x="th0_val", y="F1",
                    markers=["o"], 
                    ax=ax[0]
                )
        sns.scatterplot(data=df4,
                x='th0_val', y='F1',
                s=150,
                marker='X',
                color='g',
                label='CNN-3thresholds'
               )

        
        
        fig.tight_layout()
        
        df5 = df2.groupby(['run_name']).mean().transpose()
        
        for row in df5.index:
            if row in float_format:
                df5.loc[row] = df5.loc[row].map('{:,.2f}'.format)
            elif row in int_format:
                df5.loc[row] = df5.loc[row].map('{:,.0f}'.format)


        styler = df5.transpose().style
        styles = [dict(selector="caption", props=[("text-align", "center"),
                                              ("font-size", "120%"),
                                              ("font-weight", "bold")])]
        styler.set_caption(f"Bin of sunspots of size  {bins[i]} -> {len(binned_gt[i])} in GroudTruth").set_table_styles(styles)
        styler.set_properties(**{'text-align': 'center'})
        display(styler)
    #     display(df.transpose())

    


    

  0%|          | 0/1 [00:00<?, ?it/s]

["['T250']", "['T275']", "['T300']", "['T325']", "['T350']", "['T375']", "['T400']", "['T425']", "['T450']", "['T475']", "['T500']"]
['0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '0', '2', '3', '4', '5', '6', '7', '8', '9', '10']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
["CNN-['T500']", "CNN-['T500']", "CNN-['T500']", "CNN-['T500']", "CNN-['T500']", "CNN-['T475']", "CNN-['T475']", "CNN-['T475']", "CNN-['T475']", "CNN-['T475']", "CNN-['T450']", "CNN-['T450']", "CNN-['T450']", "CNN-['T450']", "CNN-['T450']", "CNN-['T425']", "CNN-['T425']", "CNN-['T425']", "CNN-['T425']", "CNN-['T425']", "CNN-['T400']", "CNN-['T400']", "CNN-['T400']", "CNN-['T400']", "CNN-['T400']", "CNN-['T375']", "CNN-['

/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:409: RuntimeWarning: Mean of empty slice.
  loc = data.mean()
/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:414: RuntimeWarning: Mean of empty slice.
  scale = np.sqrt(((data - loc)**2).mean())


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,F1,num_FN,percent_FN,num_FP,percent_FP,FP_area,run_number,num_th,num_tta,th0_val
run_name,,,,,,,,,,
CNN-['T300'],0.971429,0,0.00,0,0.07,430,2.800000,1.000000,0.000000,300.000000
CNN-['T325'],1.000000,0,0.00,0,0.00,0,2.800000,1.000000,0.000000,325.000000
CNN-['T350'],1.000000,0,0.00,0,0.00,0,2.800000,1.000000,0.000000,350.000000
CNN-['T375'],1.000000,0,0.00,0,0.00,0,2.800000,1.000000,0.000000,375.000000
CNN-['T400'],1.000000,0,0.00,0,0.00,0,2.800000,1.000000,0.000000,400.000000
"CNN-['T425', 'T375', 'T325']",1.000000,0,0.00,0,0.00,0,5.400000,3.000000,0.000000,375.000000
CNN-['T425'],1.000000,0,0.00,0,0.00,0,2.800000,1.000000,0.000000,425.000000
CNN-['T450'],1.000000,0,0.00,0,0.00,0,2.800000,1.000000,0.000000,450.000000
CNN-['T475'],0.971429,0,0.00,0,0.07,591,2.800000,1.000000,0.000000,475.000000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1374: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(
/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1374: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(
/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1374: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(
/home/ucl/elen/nsayez/miniconda3/envs/new-bioblue/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1374: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(


,F1,num_FN,percent_FN,num_FP,percent_FP,FP_area,run_number,num_th,num_tta,th0_val
run_name,,,,,,,,,,
CNN-['T300'],nan,0,nan,0,nan,0,2.800000,1.000000,0.000000,300.000000
CNN-['T325'],nan,0,nan,0,nan,0,2.800000,1.000000,0.000000,325.000000
CNN-['T350'],nan,0,nan,0,nan,0,2.800000,1.000000,0.000000,350.000000
CNN-['T375'],nan,0,nan,0,nan,0,2.800000,1.000000,0.000000,375.000000
CNN-['T400'],nan,0,nan,0,nan,0,2.800000,1.000000,0.000000,400.000000
"CNN-['T425', 'T375', 'T325']",nan,0,nan,0,nan,0,5.400000,3.000000,0.000000,375.000000
CNN-['T425'],nan,0,nan,0,nan,0,2.800000,1.000000,0.000000,425.000000
CNN-['T450'],nan,0,nan,0,nan,0,2.800000,1.000000,0.000000,450.000000
CNN-['T475'],nan,0,nan,0,nan,0,2.800000,1.000000,0.000000,475.000000


In [ ]:
# for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)):
#     print(c_name,c_ids)
#     for i in range(len(bins)):
#     #     print(bins[i])
#         df = pd.DataFrame.from_dict(bins_stats[c_name][i])#.astype(float)
        
#         df2 = df.append(pd.Series(show_names, index=df.columns, name='run_name'))
#         df2 = df.append(pd.Series(show_names, index=df.columns, name='run_name'))
#         df2 = df2.append(pd.Series(run_numbers, index=df.columns, name='run_number'))
#         df2 = df2.append(pd.Series(num_TH, index=df.columns, name='num_th'))
#         df2 = df2.append(pd.Series(num_ttas, index=df.columns, name='num_tta'))
#         df2 = df2.append(pd.Series([int(t[0]) for t in th_vals], index=df.columns, name='th0_val'))
        
#         df2 = df2.transpose()
#         for f in int_format:
#             df2[f]=df2[f].astype(int)
#         for f in float_format:
#             df2[f]=df2[f].astype(float)
#         for f in ['F1','th0_val','num_tta','num_th','run_number']:
#             df2[f] = pd.to_numeric(df2[f])
#         df2.index = range(len(th_vals))
#         df2["percent_FN"] = 0.01*df2["percent_FN"]
#         df2["percent_FP"] = 0.01*df2["percent_FP"]
        
        
# #         print(df2.dtypes)

# #         df2.index = [int(t[0]) for t in th_vals]
        
# #         numbers = [show_names[i] for i in index_lists[0]]


# #         tmp_df = pd.DataFrame(show_names)
# #         print(tmp_df)
# #         df2 = df.append(tmp_df.transpose())
# #         display(df2)
# #         display(df)
    
# #         df.columns = show_names
    
#         #F1 
#         F1 = df.transpose()["F1"].to_numpy()
    
#         # Roc curves
#         FN_rate = df.transpose()["percent_FN"].to_numpy()
# #         print(FN_rate)
#         FP_rate = df.transpose()["percent_FP"].to_numpy()

        
#         fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
#         g = sns.lineplot(
#                     data=df2,
#                     x="th0_val", y="F1",
# #                     markers=["o"], 
# #                     kind="line",
#                     ax=ax
#                 )
#         g = sns.lineplot(
#                     data=df2,
#                     x="th0_val", y="percent_FN",
# #                     markers=["o"], 
# #                     kind="line",
#                     ax=ax
#             )
#         g = sns.lineplot(
#                     data=df2,
#                     x="th0_val", y="percent_FP",
# #                     markers=["o"], 
# #                     kind="line",
#                     ax=ax
#             )
#         g.set_ylim([0,1])
#         fig.tight_layout()
# #         g = sns.catplot(
# #             data=df2.transpose(),
# #             x="run_name", y="F1",
# #             markers=["o"], 
# #             linestyles=["-", "--"],
# #             kind="point",
# #         )
# #         [plt.setp(ax.get_xticklabels(), rotation=90) for ax in g.axes.flat]
# #         [a.invert_xaxis() for a in g.axes.flat]
# #         [a.set_ylim([0,1]) for a in g.axes.flat]
# #         g.tight_layout()

# #         fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
# # #         sns.scatterplot(
# #         sns.boxplot(
# #                     data=df2.transpose(),
# # #                     kind="line",
# #                     x="run_name", y="F1",ax = ax
# #                 )
# #         for tick in ax.get_xticklabels():
# #             tick.set_rotation(90)
# #         fig.tight_layout()
        
# #         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(5, 5))
# #         #figure for CNN with single TH.
# #         FN_rate = np.array([FN_rate[i] for i in index_lists[0]])
# #         show_names_single_thresh = [show_names[i] for i in index_lists[0]]
# #         ax[0].set_title(f"Bin of sunspots of size  {bins[i]} -> {len(binned_gt[i])} in GroudTruth")
# #         ax[0].plot( show_names_single_thresh, FN_rate*0.01, marker='o')
# #         ax[0].plot( show_names_single_thresh, FP_rate*0.01, marker='o')
# #         ax[0].plot( show_names_single_thresh, F1, marker='o')
# #         ax[0].invert_xaxis()
# #         ax[0].legend(['FN_rate', 'FP_rate', 'F1 score'])
# #         ax[0].set_ylim([0,1])
# #         # set x_label vertically
# #         for tick in ax[0].get_xticklabels():
# #             tick.set_rotation(90)
# #         fig.tight_layout()
        
        
# #         #plot FP and FN rates along their index in a single plot, add a legend, set ticks vertically
# #         fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
# #         ax.set_title(f"Bin of sunspots of size  {bins[i]} -> {len(binned_gt[i])} in GroudTruth")
# #         ax.plot( show_names, FN_rate*0.01, marker='o')
# #         ax.plot( show_names, FP_rate*0.01, marker='o')
# #         ax.plot( show_names, F1, marker='o')
# #         ax.invert_xaxis()
# #         ax.legend(['FN_rate', 'FP_rate', 'F1 score'])
# #         ax.set_ylim([0,1])
# #         # set x_label vertically
# #         for tick in ax.get_xticklabels():
# #             tick.set_rotation(90)
# #         fig.tight_layout()

#         for row in df.index:
#             if row in float_format:
#                 df.loc[row] = df.loc[row].map('{:,.2f}'.format)
#             elif row in int_format:
#                 df.loc[row] = df.loc[row].map('{:,.0f}'.format)

#     #     df.loc["percent_missed"] = pd.Series(100* df.loc["num_missed"] / len(binned_gt[i]), dtype=int)

#         styler = df.transpose().style
#     #     pd.set_option('display.float_format',
#     #       lambda x: '{:,0f}'.format(x) if( (abs(x) % 1) != 0 )else '{:,.0f}'.format(x))
#         styles = [dict(selector="caption", props=[("text-align", "center"),
#                                               ("font-size", "120%"),
#                                               ("font-weight", "bold")])]
#         styler.set_caption(f"Bin of sunspots of size  {bins[i]} -> {len(binned_gt[i])} in GroudTruth").set_table_styles(styles)
#         styler.set_properties(**{'text-align': 'center'})
#         display(styler)
#     #     display(df.transpose())
    


    

In [108]:
# Distribution of FPs along images
def get_FP_distribution(seg_to_compare, FP_per_segmenter):
    
    distributions = {}
    
    for seg in tqdm(seg_to_compare):
        cur_FPs = FP_per_segmenter[seg]
        
#         print(cur_FPs.keys())
        
#         for c_name, c_ids in (pbar :=tqdm(all_classes.items(), leave=False)):
        cur_class_FPs = cur_FPs['foreground']
        cur_images = {}

#         print(cur_class_FPs.keys())
        for b, bin_FPs  in cur_class_FPs.items():
#             print(bin_FPs)
            for FP in bin_FPs['false_positives']:

#                 print(FP)
                if FP['image'] not in cur_images:
                    cur_images[FP['image']] = 0

                cur_images[FP['image']] += 1

#         print(cur_images)

        distributions[seg] = cur_images

    return distributions

distros = get_FP_distribution(seg_to_compare, FP_per_segmenter['predictors'])

for k,v in distros.items():
    all_im_distro = {os.path.basename(test_im).split('.')[0] :0 for test_im in gt_lst }
    for  im , num_fp in v.items():
        all_im_distro[im] += num_fp
        
    plt.figure()
    plt.title(k)
    plt.xticks(range(len(all_im_distro.keys())),rotation=90)
    plt.bar(range(len(all_im_distro.keys())), all_im_distro.values(), .5, color='g')
        
    

  0%|          | 0/24 [00:00<?, ?it/s]

KeyError: '2023_T300'

In [163]:
sns.set(style = 'whitegrid')
fmri = sns.load_dataset("fmri")
fmri

,subject,timepoint,event,region,signal
0,s13,18,stim,parietal,-0.017552
1,s5,14,stim,parietal,-0.080883
2,s12,18,stim,parietal,-0.081033
3,s11,18,stim,parietal,-0.046134
4,s10,18,stim,parietal,-0.037970
...,...,...,...,...,...
1059,s0,8,cue,frontal,0.018165
1060,s13,7,cue,frontal,-0.029130
1061,s12,7,cue,frontal,-0.004939
1062,s11,7,cue,frontal,-0.025367


In [107]:
# TOKEN + CONFUSION MATRIX

def compare_generators_with_tokens(generators_names, scores_per_generator, metric='DICE', tolerance=0.05, normalized=False):
    out_dict = {i:{} for i in range(len(bins))}
       
    for i, b in enumerate( bins[:]):
        
        bin_conf_matrix = np.zeros((len(generators_names),len(generators_names)))
        
        bin_tokens = {t:0 for t in generators_names}
            
        scores_fg_bg = {gen_name: np.array([item['value']
                                            for item in scores_per_generator[gen_name]['foreground'][str(i)][metric]
#                                             for item in scores_per_generator[gen_name][0][str(i)][metric]
#                                             for item in scores_per_generator[gen_name][0][i][metric]
                                           ]) 
                                                                    
                        for gen_name in generators_names}
        
        stacked = np.expand_dims(scores_fg_bg[list(scores_fg_bg.keys())[0]].copy(), axis=0).T
        for k in list(scores_fg_bg.keys())[1:]:
            stacked = np.hstack((stacked, np.expand_dims(scores_fg_bg[k],axis=0).T))

        
        ### Find the winners for all sunspots (winners = those with metrics are in the tolerance range from
        ### maximal value of metric)
        max_stacked = np.max(stacked, axis=1)
        print(max_stacked.shape)
        for j, m in enumerate(max_stacked):
#             winners = np.argwhere(stacked[j]== m).flatten().tolist()
            winners = np.argwhere(abs(stacked[j] - m) < tolerance).flatten().tolist()
            losers = [item for item in range(len(generators_names)) if not item in winners]
                        
            winners_names = [generators_names[w] for w in winners]

            for w in winners_names: 
                bin_tokens[w] += 1
            
            for n in range(len(generators_names)):
                if n in winners: ## all winners get 1 point 
                    for w in winners:
                        bin_conf_matrix[n,w] += 1
        if normalized:
            bin_conf_matrix /= len(scores_fg_bg[list(scores_fg_bg.keys())[0]])
                    
        for t in generators_names:         
            out_dict[i][t] = bin_tokens[t]
        out_dict[i]["confusion_matrix"] = bin_conf_matrix
#         print(out_dict)
    
    return out_dict

from copy import deepcopy

# scores_per_segmenter = deepcopy(score_per_segmenter['generators'])
# scores_per_segmenter.update(score_per_segmenter['predictors'])

# seg_to_compare = [k for k in scores_per_segmenter.keys()]
# gen_to_compare =  [k for k in score_per_segmenter['generators'].keys()]
gen_to_compare =  [
    '2023_T300',
#     '2023_T325',
    '2023_T350',
#     '2023_T375',
    '2023_T400',
#     '2023_T425',
    '2023_T450',
#     '2023_T475',
    '2023_T500',                 
    ]
gen_to_compare_show_names = [ f'T{n.split("T")[1:]}' for n in gen_to_compare]
print(gen_to_compare_show_names)

# pred_to_compare = [k for k in score_per_segmenter['predictors'].keys()]
pred_to_compare = [
    'CNN-2013-15_UNet_T500_StepLR_epoch_1_0TTA',
#    'CNN-2013-15_UNet_T475_StepLR_epoch_1_0TTA', 
   'CNN-2013-15_UNet_T450_StepLR_epoch_1_0TTA',
#    'CNN-2013-15_UNet_T425_StepLR_epoch_1_0TTA',
   'CNN-2013-15_UNet_T400_StepLR_epoch_1_0TTA', 
#    'CNN-2013-15_UNet_T375_StepLR_epoch_1_0TTA', 
   'CNN-2013-15_UNet_T350_StepLR_epoch_1_0TTA', 
#    'CNN-2013-15_UNet_T325_StepLR_epoch_1_0TTA', 
   'CNN-2013-15_UNet_T300_StepLR_epoch_1_0TTA',
    
    
]
pred_to_compare.reverse()
pred_to_compare_show_names = [ f"CNN-{n.split('_')[2:3]}" for n in pred_to_compare]

pred_to_compare = [k for k,v in score_per_segmenter['predictors'].items()]
# show_names = [f'CNN-{seg.split("_")[2:3]}' for seg in seg_to_compare]
pred_to_compare_show_names = [f'CNN-{re.search("CNN-2013-15_UNet_(.*)_StepLR*", seg).group(1).split("_")}'.replace(" ","") for seg in seg_to_compare]

print(pred_to_compare_show_names)

seg_to_compare = gen_to_compare + pred_to_compare

scores_per_segmenter = {k: deepcopy(score_per_segmenter['generators'][k]) for k in gen_to_compare}
scores_per_segmenter.update({k: deepcopy(score_per_segmenter['predictors'][k]) for k in pred_to_compare})



bins_tokens = compare_generators_with_tokens(seg_to_compare,scores_per_segmenter, metric="IoU",
                                             tolerance=0.05, normalized=True)

# print(bins_tokens[0])

def style_diag(data):
    diag_mask = pd.DataFrame("", index=data.index, columns=data.columns)
    min_axis = min(diag_mask.shape)
    diag_mask.iloc[range(min_axis), range(min_axis)] = "font-weight:bold"
    return diag_mask

for i in range(len(bins)):
#     print(bins[i])
#     col_names = [t for t in seg_to_compare]
#     col_names = [ f'{name}' for name in col_names]
    col_names = gen_to_compare_show_names + pred_to_compare_show_names
    
    df = pd.DataFrame(bins_tokens[i]["confusion_matrix"], columns=col_names, index=col_names)
    styles = [dict(selector="caption", props=[("text-align", "center"),
                                              ("font-size", "120%"),
                                              ("font-weight", "bold")])]
    styler = df.style
    styler.set_caption(f"Token matrix for bin {bins[i]}").set_table_styles(styles)
    styler.format(na_rep='MISSING',formatter="{:.2f}")
    styler.apply(style_diag, axis=None)
    styler.set_properties(**{'text-align': 'center'})
    display(styler)


["T['300']", "T['350']", "T['400']", "T['450']", "T['500']"]
["CNN-['T500']", "CNN-['T475']", "CNN-['T450']", "CNN-['T425']", "CNN-['T400']", "CNN-['T375']", "CNN-['T350']", "CNN-['T325']", "CNN-['T300']", "CNN-['T500','T450']", "CNN-['T500','T400']", "CNN-['T500','T350']", "CNN-['T500','T300']", "CNN-['T450','T400']", "CNN-['T450','T350']", "CNN-['T450','T300']", "CNN-['T400','T350']", "CNN-['T400','T300']", "CNN-['T350','T300']"]
(315,)
(90,)
(114,)
(51,)
(29,)
(16,)
(3,)
(0,)


/tmp/ipykernel_32969/3370890654.py:44: RuntimeWarning: invalid value encountered in true_divide
  bin_conf_matrix /= len(scores_fg_bg[list(scores_fg_bg.keys())[0]])


,T['300'],T['350'],T['400'],T['450'],T['500'],CNN-['T500'],CNN-['T475'],CNN-['T450'],CNN-['T425'],CNN-['T400'],CNN-['T375'],CNN-['T350'],CNN-['T325'],CNN-['T300'],"CNN-['T500','T450']","CNN-['T500','T400']","CNN-['T500','T350']","CNN-['T500','T300']","CNN-['T450','T400']","CNN-['T450','T350']","CNN-['T450','T300']","CNN-['T400','T350']","CNN-['T400','T300']","CNN-['T350','T300']"
T['300'],0.36,0.24,0.22,0.22,0.21,0.21,0.21,0.21,0.22,0.22,0.24,0.23,0.24,0.25,0.21,0.21,0.21,0.22,0.22,0.22,0.23,0.22,0.22,0.24
T['350'],0.24,0.33,0.24,0.23,0.21,0.21,0.21,0.22,0.23,0.23,0.24,0.24,0.24,0.23,0.21,0.22,0.22,0.23,0.23,0.23,0.23,0.23,0.23,0.23
T['400'],0.22,0.24,0.30,0.26,0.22,0.21,0.21,0.23,0.23,0.23,0.24,0.24,0.23,0.23,0.22,0.22,0.22,0.22,0.23,0.23,0.23,0.23,0.23,0.22
T['450'],0.22,0.23,0.26,0.30,0.24,0.22,0.22,0.23,0.23,0.22,0.23,0.24,0.22,0.23,0.23,0.23,0.23,0.22,0.23,0.23,0.23,0.22,0.22,0.21
T['500'],0.21,0.21,0.22,0.24,0.27,0.22,0.22,0.22,0.22,0.22,0.21,0.22,0.21,0.21,0.23,0.23,0.22,0.22,0.22,0.21,0.21,0.21,0.22,0.21
CNN-['T500'],0.21,0.21,0.21,0.22,0.22,0.27,0.23,0.23,0.22,0.22,0.22,0.22,0.21,0.21,0.24,0.23,0.23,0.22,0.22,0.22,0.22,0.22,0.22,0.21
CNN-['T475'],0.21,0.21,0.21,0.22,0.22,0.23,0.25,0.23,0.22,0.22,0.22,0.22,0.21,0.21,0.24,0.24,0.23,0.22,0.22,0.22,0.22,0.21,0.23,0.21
CNN-['T450'],0.21,0.22,0.23,0.23,0.22,0.23,0.23,0.27,0.23,0.24,0.23,0.22,0.22,0.23,0.23,0.25,0.26,0.24,0.23,0.24,0.23,0.23,0.24,0.22
CNN-['T425'],0.22,0.23,0.23,0.23,0.22,0.22,0.22,0.23,0.28,0.25,0.23,0.24,0.22,0.23,0.22,0.24,0.26,0.25,0.25,0.25,0.24,0.25,0.24,0.22
CNN-['T400'],0.22,0.23,0.23,0.22,0.22,0.22,0.22,0.24,0.25,0.31,0.23,0.24,0.23,0.23,0.22,0.25,0.26,0.26,0.25,0.26,0.23,0.25,0.25,0.23


,T['300'],T['350'],T['400'],T['450'],T['500'],CNN-['T500'],CNN-['T475'],CNN-['T450'],CNN-['T425'],CNN-['T400'],CNN-['T375'],CNN-['T350'],CNN-['T325'],CNN-['T300'],"CNN-['T500','T450']","CNN-['T500','T400']","CNN-['T500','T350']","CNN-['T500','T300']","CNN-['T450','T400']","CNN-['T450','T350']","CNN-['T450','T300']","CNN-['T400','T350']","CNN-['T400','T300']","CNN-['T350','T300']"
T['300'],0.40,0.18,0.04,0.03,0.02,0.03,0.03,0.02,0.02,0.02,0.02,0.02,0.07,0.10,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.03
T['350'],0.18,0.37,0.16,0.08,0.06,0.03,0.04,0.06,0.07,0.08,0.06,0.08,0.06,0.07,0.06,0.07,0.04,0.06,0.08,0.09,0.07,0.08,0.08,0.04
T['400'],0.04,0.16,0.24,0.14,0.07,0.03,0.04,0.07,0.08,0.09,0.07,0.08,0.06,0.06,0.06,0.08,0.06,0.07,0.09,0.09,0.08,0.08,0.07,0.06
T['450'],0.03,0.08,0.14,0.21,0.10,0.06,0.08,0.10,0.10,0.08,0.07,0.07,0.06,0.06,0.08,0.09,0.09,0.06,0.08,0.09,0.07,0.07,0.06,0.06
T['500'],0.02,0.06,0.07,0.10,0.14,0.07,0.08,0.11,0.09,0.07,0.06,0.06,0.06,0.04,0.09,0.10,0.08,0.06,0.08,0.06,0.07,0.06,0.07,0.04
CNN-['T500'],0.03,0.03,0.03,0.06,0.07,0.11,0.11,0.08,0.06,0.03,0.03,0.03,0.03,0.03,0.07,0.06,0.06,0.03,0.04,0.03,0.03,0.03,0.03,0.03
CNN-['T475'],0.03,0.04,0.04,0.08,0.08,0.11,0.14,0.10,0.09,0.06,0.04,0.04,0.04,0.03,0.10,0.08,0.08,0.04,0.06,0.07,0.04,0.06,0.04,0.03
CNN-['T450'],0.02,0.06,0.07,0.10,0.11,0.08,0.10,0.14,0.11,0.08,0.06,0.06,0.06,0.04,0.10,0.11,0.09,0.07,0.09,0.08,0.08,0.07,0.08,0.04
CNN-['T425'],0.02,0.07,0.08,0.10,0.09,0.06,0.09,0.11,0.17,0.10,0.06,0.07,0.07,0.04,0.10,0.10,0.10,0.08,0.12,0.11,0.08,0.08,0.07,0.04
CNN-['T400'],0.02,0.08,0.09,0.08,0.07,0.03,0.06,0.08,0.10,0.12,0.06,0.07,0.06,0.04,0.07,0.08,0.06,0.07,0.10,0.10,0.09,0.10,0.08,0.04


,T['300'],T['350'],T['400'],T['450'],T['500'],CNN-['T500'],CNN-['T475'],CNN-['T450'],CNN-['T425'],CNN-['T400'],CNN-['T375'],CNN-['T350'],CNN-['T325'],CNN-['T300'],"CNN-['T500','T450']","CNN-['T500','T400']","CNN-['T500','T350']","CNN-['T500','T300']","CNN-['T450','T400']","CNN-['T450','T350']","CNN-['T450','T300']","CNN-['T400','T350']","CNN-['T400','T300']","CNN-['T350','T300']"
T['300'],0.36,0.26,0.13,0.06,0.03,0.03,0.04,0.04,0.07,0.08,0.09,0.12,0.17,0.18,0.04,0.04,0.04,0.04,0.06,0.06,0.09,0.08,0.11,0.13
T['350'],0.26,0.46,0.26,0.14,0.05,0.03,0.03,0.06,0.12,0.13,0.17,0.20,0.21,0.15,0.04,0.07,0.09,0.07,0.11,0.11,0.16,0.15,0.17,0.16
T['400'],0.13,0.26,0.39,0.25,0.13,0.09,0.09,0.12,0.17,0.15,0.15,0.16,0.11,0.07,0.10,0.13,0.13,0.12,0.14,0.13,0.16,0.16,0.15,0.11
T['450'],0.06,0.14,0.25,0.37,0.23,0.11,0.11,0.16,0.17,0.13,0.11,0.09,0.04,0.03,0.15,0.15,0.11,0.11,0.11,0.11,0.14,0.11,0.09,0.05
T['500'],0.03,0.05,0.13,0.23,0.33,0.15,0.15,0.18,0.13,0.09,0.05,0.04,0.02,0.02,0.20,0.17,0.10,0.10,0.09,0.10,0.11,0.06,0.04,0.03
CNN-['T500'],0.03,0.03,0.09,0.11,0.15,0.21,0.19,0.18,0.14,0.09,0.04,0.04,0.02,0.02,0.20,0.18,0.12,0.11,0.11,0.11,0.10,0.05,0.04,0.02
CNN-['T475'],0.04,0.03,0.09,0.11,0.15,0.19,0.22,0.18,0.14,0.09,0.05,0.05,0.02,0.03,0.21,0.20,0.12,0.11,0.11,0.11,0.11,0.06,0.04,0.02
CNN-['T450'],0.04,0.06,0.12,0.16,0.18,0.18,0.18,0.28,0.18,0.11,0.08,0.06,0.02,0.03,0.21,0.21,0.15,0.14,0.16,0.16,0.11,0.10,0.07,0.02
CNN-['T425'],0.07,0.12,0.17,0.17,0.13,0.14,0.14,0.18,0.27,0.18,0.12,0.11,0.04,0.04,0.17,0.18,0.18,0.15,0.18,0.18,0.15,0.13,0.11,0.04
CNN-['T400'],0.08,0.13,0.15,0.13,0.09,0.09,0.09,0.11,0.18,0.24,0.11,0.11,0.05,0.04,0.10,0.14,0.14,0.14,0.17,0.16,0.15,0.16,0.12,0.06


,T['300'],T['350'],T['400'],T['450'],T['500'],CNN-['T500'],CNN-['T475'],CNN-['T450'],CNN-['T425'],CNN-['T400'],CNN-['T375'],CNN-['T350'],CNN-['T325'],CNN-['T300'],"CNN-['T500','T450']","CNN-['T500','T400']","CNN-['T500','T350']","CNN-['T500','T300']","CNN-['T450','T400']","CNN-['T450','T350']","CNN-['T450','T300']","CNN-['T400','T350']","CNN-['T400','T300']","CNN-['T350','T300']"
T['300'],0.43,0.41,0.25,0.20,0.08,0.04,0.06,0.08,0.10,0.12,0.24,0.31,0.29,0.31,0.04,0.08,0.08,0.14,0.14,0.14,0.10,0.22,0.25,0.27
T['350'],0.41,0.61,0.43,0.35,0.22,0.08,0.12,0.16,0.20,0.25,0.39,0.43,0.41,0.35,0.10,0.18,0.18,0.27,0.27,0.29,0.25,0.35,0.39,0.37
T['400'],0.25,0.43,0.59,0.51,0.33,0.16,0.16,0.24,0.31,0.37,0.45,0.37,0.33,0.25,0.18,0.25,0.31,0.39,0.37,0.43,0.31,0.43,0.41,0.31
T['450'],0.20,0.35,0.51,0.59,0.41,0.18,0.16,0.27,0.37,0.37,0.37,0.29,0.27,0.20,0.22,0.25,0.33,0.41,0.39,0.41,0.29,0.37,0.35,0.24
T['500'],0.08,0.22,0.33,0.41,0.45,0.20,0.18,0.25,0.31,0.31,0.27,0.20,0.20,0.12,0.24,0.24,0.27,0.35,0.35,0.35,0.25,0.27,0.24,0.16
CNN-['T500'],0.04,0.08,0.16,0.18,0.20,0.27,0.24,0.24,0.20,0.18,0.16,0.12,0.10,0.06,0.22,0.24,0.22,0.22,0.20,0.24,0.16,0.16,0.12,0.08
CNN-['T475'],0.06,0.12,0.16,0.16,0.18,0.24,0.27,0.24,0.22,0.20,0.18,0.14,0.10,0.10,0.22,0.25,0.24,0.24,0.22,0.25,0.16,0.18,0.14,0.10
CNN-['T450'],0.08,0.16,0.24,0.27,0.25,0.24,0.24,0.39,0.35,0.33,0.25,0.24,0.24,0.10,0.27,0.31,0.33,0.29,0.35,0.35,0.25,0.27,0.24,0.16
CNN-['T425'],0.10,0.20,0.31,0.37,0.31,0.20,0.22,0.35,0.47,0.41,0.33,0.29,0.25,0.16,0.25,0.29,0.41,0.39,0.45,0.45,0.31,0.37,0.33,0.20
CNN-['T400'],0.12,0.25,0.37,0.37,0.31,0.18,0.20,0.33,0.41,0.51,0.39,0.35,0.31,0.20,0.24,0.29,0.39,0.43,0.45,0.49,0.35,0.43,0.39,0.24


,T['300'],T['350'],T['400'],T['450'],T['500'],CNN-['T500'],CNN-['T475'],CNN-['T450'],CNN-['T425'],CNN-['T400'],CNN-['T375'],CNN-['T350'],CNN-['T325'],CNN-['T300'],"CNN-['T500','T450']","CNN-['T500','T400']","CNN-['T500','T350']","CNN-['T500','T300']","CNN-['T450','T400']","CNN-['T450','T350']","CNN-['T450','T300']","CNN-['T400','T350']","CNN-['T400','T300']","CNN-['T350','T300']"
T['300'],0.59,0.55,0.48,0.31,0.17,0.14,0.17,0.28,0.34,0.38,0.52,0.52,0.55,0.48,0.17,0.21,0.34,0.34,0.34,0.38,0.38,0.45,0.48,0.45
T['350'],0.55,0.69,0.62,0.45,0.28,0.24,0.31,0.41,0.48,0.52,0.66,0.66,0.66,0.52,0.28,0.34,0.48,0.48,0.48,0.52,0.48,0.59,0.62,0.52
T['400'],0.48,0.62,0.76,0.55,0.38,0.38,0.45,0.55,0.62,0.62,0.62,0.66,0.59,0.45,0.41,0.48,0.62,0.59,0.55,0.62,0.55,0.66,0.62,0.48
T['450'],0.31,0.45,0.55,0.55,0.38,0.34,0.41,0.52,0.52,0.48,0.45,0.48,0.41,0.28,0.38,0.45,0.52,0.52,0.52,0.52,0.45,0.48,0.48,0.34
T['500'],0.17,0.28,0.38,0.38,0.45,0.38,0.41,0.41,0.45,0.34,0.31,0.31,0.24,0.14,0.38,0.41,0.45,0.41,0.45,0.45,0.34,0.34,0.34,0.21
CNN-['T500'],0.14,0.24,0.38,0.34,0.38,0.45,0.45,0.45,0.45,0.38,0.28,0.31,0.21,0.17,0.45,0.45,0.45,0.38,0.41,0.45,0.31,0.38,0.31,0.21
CNN-['T475'],0.17,0.31,0.45,0.41,0.41,0.45,0.52,0.52,0.52,0.45,0.34,0.38,0.28,0.17,0.45,0.52,0.52,0.45,0.48,0.52,0.38,0.45,0.38,0.24
CNN-['T450'],0.28,0.41,0.55,0.52,0.41,0.45,0.52,0.62,0.62,0.55,0.45,0.48,0.38,0.28,0.48,0.55,0.62,0.55,0.59,0.62,0.45,0.55,0.48,0.34
CNN-['T425'],0.34,0.48,0.62,0.52,0.45,0.45,0.52,0.62,0.72,0.62,0.55,0.55,0.45,0.34,0.48,0.55,0.69,0.66,0.66,0.72,0.52,0.62,0.59,0.38
CNN-['T400'],0.38,0.52,0.62,0.48,0.34,0.38,0.45,0.55,0.62,0.66,0.55,0.59,0.48,0.38,0.41,0.48,0.62,0.59,0.55,0.62,0.48,0.66,0.59,0.41


,T['300'],T['350'],T['400'],T['450'],T['500'],CNN-['T500'],CNN-['T475'],CNN-['T450'],CNN-['T425'],CNN-['T400'],CNN-['T375'],CNN-['T350'],CNN-['T325'],CNN-['T300'],"CNN-['T500','T450']","CNN-['T500','T400']","CNN-['T500','T350']","CNN-['T500','T300']","CNN-['T450','T400']","CNN-['T450','T350']","CNN-['T450','T300']","CNN-['T400','T350']","CNN-['T400','T300']","CNN-['T350','T300']"
T['300'],0.69,0.62,0.44,0.25,0.12,0.00,0.00,0.06,0.06,0.25,0.44,0.38,0.62,0.62,0.06,0.06,0.19,0.19,0.19,0.12,0.19,0.44,0.44,0.62
T['350'],0.62,0.88,0.69,0.50,0.25,0.06,0.06,0.19,0.12,0.38,0.44,0.44,0.69,0.69,0.12,0.12,0.31,0.31,0.31,0.25,0.25,0.50,0.50,0.62
T['400'],0.44,0.69,0.69,0.50,0.25,0.06,0.06,0.19,0.12,0.38,0.38,0.44,0.56,0.50,0.12,0.12,0.31,0.31,0.31,0.25,0.25,0.50,0.50,0.44
T['450'],0.25,0.50,0.50,0.50,0.25,0.06,0.06,0.19,0.12,0.25,0.19,0.25,0.38,0.31,0.12,0.12,0.25,0.25,0.19,0.25,0.19,0.31,0.31,0.25
T['500'],0.12,0.25,0.25,0.25,0.25,0.06,0.06,0.12,0.06,0.12,0.06,0.06,0.12,0.06,0.12,0.12,0.19,0.19,0.12,0.12,0.06,0.19,0.12,0.12
CNN-['T500'],0.00,0.06,0.06,0.06,0.06,0.12,0.12,0.12,0.00,0.06,0.06,0.00,0.00,0.00,0.12,0.12,0.12,0.12,0.12,0.12,0.06,0.06,0.00,0.00
CNN-['T475'],0.00,0.06,0.06,0.06,0.06,0.12,0.12,0.12,0.00,0.06,0.06,0.00,0.00,0.00,0.12,0.12,0.12,0.12,0.12,0.12,0.06,0.06,0.00,0.00
CNN-['T450'],0.06,0.19,0.19,0.19,0.12,0.12,0.12,0.25,0.12,0.19,0.19,0.12,0.12,0.12,0.19,0.19,0.25,0.25,0.25,0.25,0.19,0.19,0.12,0.12
CNN-['T425'],0.06,0.12,0.12,0.12,0.06,0.00,0.00,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.06,0.06,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12
CNN-['T400'],0.25,0.38,0.38,0.25,0.12,0.06,0.06,0.19,0.12,0.38,0.31,0.31,0.31,0.31,0.12,0.12,0.19,0.19,0.25,0.25,0.19,0.38,0.31,0.31


,T['300'],T['350'],T['400'],T['450'],T['500'],CNN-['T500'],CNN-['T475'],CNN-['T450'],CNN-['T425'],CNN-['T400'],CNN-['T375'],CNN-['T350'],CNN-['T325'],CNN-['T300'],"CNN-['T500','T450']","CNN-['T500','T400']","CNN-['T500','T350']","CNN-['T500','T300']","CNN-['T450','T400']","CNN-['T450','T350']","CNN-['T450','T300']","CNN-['T400','T350']","CNN-['T400','T300']","CNN-['T350','T300']"
T['300'],0.67,0.67,0.67,0.67,0.00,0.33,0.33,0.67,0.67,0.67,0.33,0.67,0.33,0.33,0.33,0.67,0.67,0.67,0.67,0.67,0.67,0.67,0.67,0.33
T['350'],0.67,1.00,1.00,1.00,0.00,0.33,0.33,1.00,1.00,1.00,0.33,1.00,0.33,0.67,0.33,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.33
T['400'],0.67,1.00,1.00,1.00,0.00,0.33,0.33,1.00,1.00,1.00,0.33,1.00,0.33,0.67,0.33,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.33
T['450'],0.67,1.00,1.00,1.00,0.00,0.33,0.33,1.00,1.00,1.00,0.33,1.00,0.33,0.67,0.33,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.33
T['500'],0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
CNN-['T500'],0.33,0.33,0.33,0.33,0.00,0.33,0.33,0.33,0.33,0.33,0.00,0.33,0.00,0.00,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.00
CNN-['T475'],0.33,0.33,0.33,0.33,0.00,0.33,0.33,0.33,0.33,0.33,0.00,0.33,0.00,0.00,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.00
CNN-['T450'],0.67,1.00,1.00,1.00,0.00,0.33,0.33,1.00,1.00,1.00,0.33,1.00,0.33,0.67,0.33,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.33
CNN-['T425'],0.67,1.00,1.00,1.00,0.00,0.33,0.33,1.00,1.00,1.00,0.33,1.00,0.33,0.67,0.33,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.33
CNN-['T400'],0.67,1.00,1.00,1.00,0.00,0.33,0.33,1.00,1.00,1.00,0.33,1.00,0.33,0.67,0.33,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.33


,T['300'],T['350'],T['400'],T['450'],T['500'],CNN-['T500'],CNN-['T475'],CNN-['T450'],CNN-['T425'],CNN-['T400'],CNN-['T375'],CNN-['T350'],CNN-['T325'],CNN-['T300'],"CNN-['T500','T450']","CNN-['T500','T400']","CNN-['T500','T350']","CNN-['T500','T300']","CNN-['T450','T400']","CNN-['T450','T350']","CNN-['T450','T300']","CNN-['T400','T350']","CNN-['T400','T300']","CNN-['T350','T300']"
T['300'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
T['350'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
T['400'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
T['450'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
T['500'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
CNN-['T500'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
CNN-['T475'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
CNN-['T450'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
CNN-['T425'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
CNN-['T400'],MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING


# Poubelle

In [34]:
a = np.array([0,1])
b = np.random.randint(5, size=(2, 4))
b

array([[1, 1, 1, 0],
       [1, 0, 4, 2]])

In [40]:

c=np.zeros_like(b)
for i in a:
    print()
    c[b == i] = 1
    print(c)
b


[[0 0 0 1]
 [0 1 0 0]]

[[1 1 1 1]
 [1 1 0 0]]


array([[1, 1, 1, 0],
       [1, 0, 4, 2]])

In [63]:
x = 1
str(str(x))

'1'